In [1]:
import pandas as pd
import numpy as np

# df=pd.read_csv('./SMSSpamCollection',sep='\t', header=None,na_filter=False,names=['class','text'])

# df['class'] = df["class"].astype('category')

# df['true_label'] = df['class'].cat.codes

# np.random.seed(1234)
# msk = np.random.rand(len(df)) < 0.66

# train_df = df[msk]
# test_df = df[~msk]

# true_labels = test_df['true_label'].tolist()

# train_df.to_pickle("train_df")
# test_df.to_pickle("test_df")

In [5]:
train_df = pd.read_pickle("train_df")
# train_df.head()
train_df.groupby("class").agg({"text": np.count_nonzero})

,text
class,
ham,3193
spam,507


In [4]:
test_df = pd.read_pickle("test_df")

true_labels = test_df['true_label'].tolist()

test_df.groupby("class").agg({"text": np.count_nonzero})


,text
class,
ham,1632
spam,240


In [171]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [105]:
def read_words(words_file):
    return [line for line in open(words_file, 'r') ]

words = read_words("blacklist.txt")

spam = [word.strip('-_\n/') for word in words]
spam = [word.replace('-',' ') for word in spam]
print(len(spam))


# l1 = []
# l2 = []
# l3 = []
# for i,w in enumerate(spam):
#     if(i%3==0):
#         l1.append(w)
#     elif(i%3==1):
#         l2.append(w)
#     else:
#         l3.append(w)
        
l1 = set()
l2 = set()
l3 = set()
for i,w in enumerate(spam):
    if(i%3==0):
        l1.add(w)
    elif(i%3==1):
        l2.add(w)
    else:
        l3.add(w)
print(len(l1),len(l2),len(l3))

28294
8846 8879 8861


In [263]:
####### Discrete ##########


# Helper function to get last name

import re
def ltp(x):
    return '(' + '|'.join(x) + ')'

# def l1words(c):
#     return (1,1) if re.search(ltp(l1), c['text'], flags=re.I) else (0,0)

# def l2words(c):
#      return (1,1) if re.search(ltp(l2), c['text'], flags=re.I) else (0,0)

# def l3words(c):
#      return (1,1) if re.search(ltp(l3), c['text'], flags=re.I) else (0,0)

# notFree = ['you','toll','your','call','meet','talk','freez']

# def notFreeSpam(c):
#     return (-1,1) if re.search('(free.*'+ltp(notFree)+')|('+ltp(notFree)+'.*free)',\
#                                flags=re.I) else (0,0)

def l1words(c):
    return (1,1) if len(l1.intersection(c['text'].split())) > 0 else (0,0)

def l2words(c):
    return (1,1) if len(l2.intersection(c['text'].split())) > 0 else (0,0)

def l3words(c):
    return (1,1) if len(l3.intersection(c['text'].split())) > 0 else (0,0)



notFree1 = {'toll','Toll','freely','call','meet','talk','feedback'}

def notFreeSpam(c):
    return (-1,1) if 'free' in c['text'].split() and len(notFree.intersection(c['text'].split()))>0 else (0,0)        

notFree2 = {'not free','you are','when','wen'}

def notFreeSpam2(c):
    return (-1,1) if 'free' in c['text'].split() and re.search(ltp(notFree2),c['text'], flags= re.I) else (0,0)        

person1 = {'I','i','u','you','ur','your','our','we','us','you\'re,'}

person2 = {'He','he','She','she','they','They','Them','them','their','Their'}

def personWords(c):
    return (-1,1) if 'free' in c['text'].split() and len(person1.intersection(c['text'].split()))>0 else (0,0)        

def secondPersonWords(c):
    return (-1,1) if 'free' in c['text'].split() and len(person2.intersection(c['text'].split()))>0 else (0,0)        

def noOfCapChars(c):
    return (1,1) if (sum(1 for ch in c['text'] if ch.isupper()) > 6) else (0,0)

LFs = [
   l1words,l2words,l3words,noOfCapChars,notFreeSpam,notFreeSpam2,personWords,secondPersonWords
]

LF_l = [1,1,1,1,-1,-1,-1,-1]


print(len(LFs),len(LF_l))

8 8


In [11]:
##### Continuous ################



def l1words(c):
    sc = 0
    word_vectors = get_word_vectors(c['text'].split())
    l1 = ['free','credit','cheap','apply','buy','attention','shop','sex','soon','now','spam']
    for w in l1:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)

def l2words(c):
    sc = 0
    l2 = ['gift','click','new','online','discount','earn','miss','hesitate','exclusive','urgent']
    word_vectors = get_word_vectors(c['text'].split())
    for w in l2:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)

def l3words(c):
    sc = 0
    l3 = ['cash','refund','insurance','money','guaranteed','save','win','teen','weight','hair']
    word_vectors = get_word_vectors(c['text'].split())
    for w in l3:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)
   
def notFreeSpam(c):
    sc = 0
    notFree = ['not','when','call','meet','talk','feedback','toll']
    word_vectors = get_word_vectors(c['text'].split())
    for w in notFree:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)
   

def notFreeSpam2(c):
    sc = 0
    notFree2 =  ['not free','you are','when']
    word_vectors = get_word_vectors(c['text'].split())
    for w in notFree2:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)

def personWords(c):
    sc = 0
    notFree2 = ['I','you','your','we','us']
    word_vectors = get_word_vectors(c['text'].split())
    for w in person1:
        sc=max(sc,get_similarity(word_vectors,w))
    return (-1,sc)
    return (-1,1) if 'free' in c['text'].split() and re.search(ltp(notFree2),c['text'], flags= re.I) else (0,0)        

def secondPersonWords(c):
    sc = 0
    notFree2 = ['he','she','they','them','their']
    word_vectors = get_word_vectors(c['text'].split())
    for w in person1:
        sc=max(sc,get_similarity(word_vectors,w))
    return (-1,sc)
   
    
def noOfCapChars(c):
    l = sum(1 for ch in c['text'] if ch.isupper()) 
    return (1,l/150)

LFs = [
   l1words,l2words,l3words,noOfCapChars,notFreeSpam,notFreeSpam2,personWords,secondPersonWords
]

LF_l = [1,1,1,1,-1,-1,-1,-1]

print(len(LFs),len(LF_l))

8 8


In [254]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''


def get_L_S_Tensor(df,msg):     
    L_S = []
    print('labelling ',msg,' data')
    for i in range(len(df.index)):
        L_S_ci=[]
        L=[]
        S=[]
        P_ik = []
        for LF in LFs:
#             print(i,LF.__name__)    
#             print(df.iloc[i]['text'])
            l,s = LF(df.iloc[i])
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%250==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
        
    return L_S


In [264]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))


test_L_S = get_L_S_Tensor(test_df,'regex test')
np.save("test_L_S_discrete",np.array(test_L_S))

train_L_S = get_L_S_Tensor(train_df,'regex train')
np.save("train_L_S_discrete",np.array(train_L_S))



# test_L_S = get_L_S_Tensor(test_df,'regex test')
# np.save("test_L_S_smooth",np.array(test_L_S))

# train_L_S = get_L_S_Tensor(train_df,'regex train')
# np.save("train_L_S_smooth",np.array(train_L_S))



print("--- %s seconds ---" % (time.time() - start_time))

# test_L_S = get_L_S_Tensor(test_cands)
# pkl.dump(test_L_S,open("test_L_S.p","wb"))

started at: 9-6-2018, 20:24:46
labelling  regex test  data
250data points labelled in 0.004436842600504557 mins
500data points labelled in 0.008656620979309082 mins
750data points labelled in 0.01289513111114502 mins
1000data points labelled in 0.017128411928812662 mins
1250data points labelled in 0.021420172850290933 mins
1500data points labelled in 0.02580949068069458 mins
1750data points labelled in 0.0302552858988444 mins
labelling  regex train  data
250data points labelled in 0.03731296062469482 mins
500data points labelled in 0.041551840305328366 mins
750data points labelled in 0.045717823505401614 mins
1000data points labelled in 0.04994643131891886 mins
1250data points labelled in 0.054119853178660075 mins
1500data points labelled in 0.05829683542251587 mins
1750data points labelled in 0.06246569554011027 mins
2000data points labelled in 0.06664372682571411 mins
2250data points labelled in 0.07073365052541097 mins
2500data points labelled in 0.0749161958694458 mins
2750data poi

In [49]:
LF_l = [1,1,1,1,-1,-1,-1,-1]

def merge(a,b):
    c = []
    for i in range(len(a)):
        ci = []
        ci_l = a[i,0,:].tolist()+b[i,0,:].tolist()
        ci_s = a[i,1,:].tolist()+b[i,1,:].tolist()
        ci.append(ci_l)
        ci.append(ci_s)
        c.append(ci)
    return c
import numpy as np
test_L_S_s = np.load("test_L_S_smooth.npy")
train_L_S_s = np.load("train_L_S_smooth.npy")

test_L_S_d = np.load("test_L_S_discrete.npy")
train_L_S_d = np.load("train_L_S_discrete.npy")

test_L_S = np.array(merge(test_L_S_d,test_L_S_s))
train_L_S = np.array(merge(train_L_S_d,train_L_S_s))

dev_L_S = test_L_S
gold_labels_dev = true_labels 


LF_l = LF_l + LF_l
print(len(LF_l))
NoOfLFs= len(LF_l)
NoOfClasses = 2
# LF_names = [lf.__name__ for lf in LFs] + ['s'+lf.__name__ for lf in LFs]
# print(len(LF_names))
print(test_L_S.shape,train_L_S.shape)

16
(1872, 2, 16) (3700, 2, 16)


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(true_labels,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(true_labels,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(true_labels, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [12]:
import numpy as np
test_L_S = np.load("test_L_S_discrete.npy")
train_L_S = np.load("train_L_S_discrete.npy")
dev_L_S = test_L_S
gold_labels_dev = true_labels 
print(test_L_S.shape,train_L_S.shape)
# LF_names= [lf.__name__ for lf in LFs]

(1872, 2, 8) (3700, 2, 8)


In [42]:
import numpy as np
test_L_S = np.load("test_L_S_smooth.npy")
train_L_S = np.load("train_L_S_smooth.npy")
dev_L_S = test_L_S
gold_labels_dev = true_labels 
print(test_L_S.shape,train_L_S.shape)

# LF_names= ['s'+lf.__name__ for lf in LFs]

(1872, 2, 8) (3700, 2, 8)


In [46]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [45]:
LF_l = [1,1,1,1,-1,-1,-1,-1]
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

8


In [40]:

def train(lr,ep,th,af,batch_size=32,LF_acc=None,LF_rec=None,pcl=np.array([-1,1],dtype=np.float64),norm=True,\
          smooth=True,penalty=0,p3k=3,alp=1,Gamma=1.0,debug=True):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (to set a max value for alpha)
    ## norm : use normalization or not
    ## Gamma : penalty tuning parameter
    
    BATCH_SIZE = batch_size
    tf.reset_default_graph()
    

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(len(dev_L_S))
#         test_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)
#         test_in`it_op = iterator.make_initializer(test_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)
        
        

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    
        g = tf.convert_to_tensor(Gamma, dtype=tf.float64)
        
        if(penalty in [4,5,7,8,9,10,11]):
            LF_a = tf.convert_to_tensor(LF_acc, dtype=tf.float64)
        
        if(penalty in [6,7,11,12]):
            LF_r = tf.convert_to_tensor(LF_rec, dtype=tf.float64)
        
        if(debug):
            print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
        if(debug):
            print("s",s)
            print("l",l)
#         print(s.graph)
        if(smooth):
            s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
            if(debug):
                print("s_",s_)

       
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            if(debug):
                print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
        else:
            pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

        if(debug):
            print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
        if(debug):
            print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        if(debug):
            print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            if(debug):
                print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            if(debug):
                print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="zy")
            
        ### for precision and recall t_pout
        def pr_t_pout(j):
            Lj = tf.map_fn(lambda li : tf.gather(li,j),l)
            if(debug):
                print("sft Lj",Lj)
            kj = tf.gather(k,j)
            if(debug):
                print("sft kj",kj)
            indices = tf.where(tf.equal(Lj,kj))
            if(debug):
                print("sft indices",indices)
            li_lij_eq_kj = tf.gather(l,tf.squeeze(indices,1))
            if(smooth):
                si_lij_eq_kj = tf.gather(s_,tf.squeeze(indices,1))
            if(debug):
                print("sft l_ij_eq_kj",li_lij_eq_kj)
            if(smooth):
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="prec_zy"))
            else:
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="prec_zy"))
            if(debug):
                print("prec_z",prec_z)
            if(smooth):
                prec_t_pout = (tf.matmul(li_lij_eq_kj*si_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            else:
                prec_t_pout = (tf.matmul(li_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            if(debug):
                print("prec_t_pout",prec_t_pout)
            return prec_t_pout
           
        def softplus_p(j):
            aj = tf.gather(LF_a,j)
            if(debug):
                print("sft aj",aj)
            f_p =  tf.reduce_sum(aj - pr_t_pout(j))
            if(debug):
                print("f_p",f_p)
            sft_p = tf.nn.softplus(f_p,name="sft_p")
            if(debug):
                print("sft_p",sft_p)
            return sft_p
        
        def softplus_r(j):
            rj = tf.gather(LF_r,j)
            if(debug):
                print("sft aj",rj)
            f_r =  rj - tf.reduce_sum( pr_t_pout(j)) 
            if(debug):
                print("f_r",f_r)
            sft_r = tf.nn.softplus(f_r,name="sft_r")
            if(debug):
                print("sft_r",sft_r)
            return sft_r
            
        
#         logsft = tf.map_fn(lambda j: tf.log(softplus(j)),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
#         sft  =  tf.map_fn(lambda j: softplus(j),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
        
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        if(debug):
            print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        if(debug):
            print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        if(debug):
            print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +(g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -(g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +(g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-pk))))
        elif(penalty == 4):
            print("precision penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 5):
            print("precision log(softplus) penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: tf.log(softplus_p(j)),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 6):
            print("recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 7):
            print("precision and recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 8):
            print("precision and sign 1 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 9):
            print("precision and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 10):
            print("precision and sign 3 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 11):
            print("precision and recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 12):
            print("recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        else:
            print("normalized loss")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
       
        if(debug):
            print("loss",loss)
        tf.summary.scalar('loss', loss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        if(debug):
            print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(loss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,loss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)
                print("dev set")
                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_dev,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()
#                 print("test set")
#                 sess.run(test_init_op)
#                 a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#                 unique, counts = np.unique(pl, return_counts=True)
#                 print(dict(zip(unique, counts)))
#                 print("acc",accuracy_score(gold_labels_test,pl))
#                 print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(pl),average="binary"))
#                 print()
                
#             # Initialize an iterator over the validation dataset.
#             sess.run(dev_init_op)
#             a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#             print(a)
#             print(t)

#             unique, counts = np.unique(pl, return_counts=True)
#             print(dict(zip(unique, counts)))

#             print("acc",accuracy_score(true_labels,pl))

# #             predictAndPrint(pl)
#             print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [54]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.85*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [64,128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

[3145. 3145. 3145. 3145. 3145. 3145. 3145. 3145. 3145. 3145. 3145. 3145.
 3145. 3145. 3145. 3145.]
batch-size: 64 alpha-init: 0.6
recall penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 2959146.3576840125
dev set
[0.59966419 0.59662467 0.59861625 0.59852027 0.60260125 0.60197537
 0.60325263 0.60187818 0.5999139  0.59909635 0.59828474 0.60192128
 0.59827266 0.60234796 0.60196187 0.6019761 ]
[[1.11906896 0.81573464 1.01104918 1.00212258 1.10485233 1.03977548
  1.16764946 1.02963845 1.14787911 1.06627299 0.98509706 1.06512126
  0.98392353 1.07701838 1.03926376 1.0406874 ]]
{0: 273, 1: 1599}
acc 0.27297008547008544
(0.14946841776110067, 0.9958333333333333, 0.25992387166938546, None)

1 loss 2959123.0325994184
dev set
[0.59817455 0.5950781  0.59712625 0.59703654 0.60412528 0.60350861
 0.6047882  0.60341265 0.59833777 0.59751978 0.59670813 0.60320012
 0.59669131 0.60388211 0.60353535 0.60354959]
[[1.1205784  0.81728596 1.01255366 1.00362401 1.10333518 1.03823884
  1.16611921 1.0281014  1.14945499 1.0678478  0.98666936 1.06359618
  0.9855036  1.07548098 1.03769335 1.039117  ]]
{0: 268, 1: 1604}
acc 0.2702991452991453
(0.14900249376558602, 0.9958333333333333, 0.2592190

2 loss 2959561.089640687
dev set
[0.8968642  0.89385698 0.89604117 0.89562734 0.90517048 0.90481632
 0.90609141 0.90477847 0.89812491 0.89806619 0.89732983 0.90512775
 0.89978897 0.90519906 0.89620948 0.89622385]
[[1.12192333 0.81854768 1.01368218 1.00505974 1.10258787 1.03715443
  1.16502358 1.02690231 1.15019108 1.06774886 0.98652217 1.06203516
  0.98408477 1.07437962 1.04539266 1.0468163 ]]
{0: 1096, 1: 776}
acc 0.6677350427350427
(0.2538659793814433, 0.8208333333333333, 0.38779527559055116, None)

3 loss 2959560.0940966345
dev set
[0.89542665 0.89240677 0.89465921 0.89414748 0.90653625 0.90620548
 0.90752079 0.9061793  0.89699278 0.89723634 0.8964945  0.90659894
 0.89981159 0.9065929  0.89481017 0.89482459]
[[1.12340274 0.82003148 1.01510454 1.00657799 1.10132299 1.03586572
  1.16367471 1.02558885 1.15150114 1.06874244 0.98752921 1.06056064
  0.984638   1.07308368 1.04693512 1.04835876]]
{0: 1092, 1: 780}
acc 0.6655982905982906
(0.25256410256410255, 0.8208333333333333, 0.3862745098

3 loss 1500122.9231608228
dev set
[0.69806376 0.69502756 0.69704002 0.69693575 0.70414473 0.703504
 0.70479523 0.70340623 0.69831636 0.69750284 0.6966877  0.70373177
 0.69667247 0.7038776  0.70355255 0.70356678]
[[1.12067176 0.81733623 1.01263207 1.00371173 1.10330967 1.03824842
  1.1661095  1.02811396 1.14946976 1.0678262  0.98661657 1.06351687
  0.98551854 1.07549032 1.03767792 1.03910157]]
{0: 571, 1: 1301}
acc 0.42788461538461536
(0.180630284396618, 0.9791666666666666, 0.30499675535366644, None)

4 loss 1500117.0488815666
dev set
[0.69728363 0.69423146 0.69626433 0.69616196 0.70490503 0.70426035
 0.70555884 0.70416164 0.69749868 0.69668732 0.69586402 0.70450861
 0.69585005 0.70463464 0.70436237 0.70437661]
[[1.12145915 0.81813597 1.01341456 1.00449279 1.10254961 1.03749041
  1.16534633 1.02735789 1.15028367 1.06863259 0.98741897 1.06273831
  0.98634253 1.07473177 1.03686726 1.03829091]]
{0: 557, 1: 1315}
acc 0.4204059829059829
(0.17870722433460076, 0.9791666666666666, 0.30225080385

...

1 loss 443419.99303479184
dev set
[0.70075118 0.69772254 0.69967552 0.69958717 0.70148481 0.7008348
 0.70211423 0.70073425 0.70104329 0.70022714 0.69941578 0.70108584
 0.69940403 0.70120723 0.70082866 0.70084289]
[[1.11796882 0.81463425 1.0099799  1.00104371 1.10595702 1.04091717
  1.16878393 1.03078538 1.14675563 1.06514047 0.98398623 1.06619361
  0.98280133 1.0781602  1.0403922  1.04181585]]
{0: 610, 1: 1262}
acc 0.44871794871794873
(0.1862123613312203, 0.9791666666666666, 0.31291611185086554, None)

2 loss 443418.57382909744
dev set
[0.70054281 0.69751635 0.69946881 0.69938229 0.7016913  0.70104124
 0.7023213  0.70094127 0.70083627 0.7000203  0.699209   0.7012926
 0.69919711 0.70141369 0.7010357  0.70104994]
[[1.11817722 0.81484054 1.01018673 1.00124887 1.10575052 1.04071072
  1.16857688 1.03057813 1.14696307 1.06534504 0.98419098 1.06598683
  0.98300798 1.07795374 1.04018525 1.0416089 ]]
{0: 609, 1: 1263}
acc 0.4481837606837607
(0.18606492478226444, 0.9791666666666666, 0.3127079174

2 loss 443594.3096623282
dev set
[1.00183684 0.99750087 1.00074284 1.00026389 1.00172718 1.00109064
 1.00234625 1.00098934 1.00212035 1.00131984 0.99922106 1.0013383
 0.99919961 1.00145511 1.00108543 1.00109951]
[[1.1180018  0.81486673 1.00958875 1.00062461 1.10679919 1.04147337
  1.16976866 1.03130212 1.14686128 1.06496652 0.98420175 1.06686634
  0.98301999 1.07889027 1.04094504 1.04237479]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

3 loss 443594.3089253701
dev set
[1.0020809  0.9972786  1.00100826 1.00050728 1.00197327 1.0013515
 1.00258275 1.00125235 1.00235984 1.00157511 0.99900273 1.00159311
 0.99897731 1.00170707 1.00134642 1.00136017]
[[1.11822277 0.81510139 1.00965584 1.00062959 1.10701506 1.04161446
  1.17000038 1.03142254 1.14709014 1.06514557 0.9844452  1.06704785
  0.98326701 1.07908545 1.04108513 1.0425176 ]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347

3 loss 242228.30931963224
dev set
[0.80074695 0.79771733 0.79967419 0.79958027 0.80148748 0.80083849
 0.80211875 0.80073837 0.80103893 0.80029666 0.79946437 0.80109074
 0.79941383 0.80121091 0.80065465 0.80066935]
[[1.11797298 0.81463944 1.0099815  1.00105031 1.10595459 1.04091328
  1.16877857 1.03078086 1.14675864 1.0651202  0.98395898 1.06618992
  0.98277166 1.07815633 1.04123552 1.04265916]]
{0: 958, 1: 914}
acc 0.6068376068376068
(0.2286652078774617, 0.8708333333333333, 0.3622183708838822, None)

4 loss 242228.17263506242
dev set
[0.80063995 0.79761118 0.79956892 0.7994739  0.80159291 0.80094433
 0.80222508 0.80084454 0.80093248 0.80021412 0.79937267 0.80119666
 0.79931079 0.80131675 0.80071435 0.80072916]
[[1.11807997 0.81474563 1.01008689 1.00115671 1.10584925 1.04080737
  1.168672   1.03067446 1.14686436 1.06521726 0.98405501 1.06608427
  0.98286773 1.07805043 1.04134009 1.04276373]]
{0: 957, 1: 915}
acc 0.6063034188034188
(0.2284153005464481, 0.8708333333333333, 0.3619047619047

4 loss 141234.19031644007
dev set
[0.60090586 0.59787461 0.59982752 0.59973783 0.60133206 0.60068165
 0.60196125 0.60058076 0.60119532 0.60037918 0.59956764 0.60093485
 0.59955574 0.60105408 0.60067624 0.60069048]
[[1.11781414 0.8144821  1.00982775 1.00089272 1.10610955 1.04107024
  1.16893652 1.03093897 1.14660299 1.06499148 0.98383839 1.06634667
  0.98264966 1.07831328 1.04054421 1.04196785]]
{0: 285, 1: 1587}
acc 0.2783119658119658
(0.14996849401386264, 0.9916666666666667, 0.26053639846743293, None)

batch-size: 2048 alpha-init: 0.7
recall penalty
0 loss 141501.9115323114
dev set
[0.70112151 0.69808853 0.70004192 0.69995252 0.70111753 0.70046712
 0.70174595 0.7003659  0.70140973 0.70059364 0.6997821  0.70071968
 0.69977022 0.70083955 0.70046178 0.70047602]
[[1.11759855 0.81426815 1.00961333 1.00067803 1.10632412 1.04128482
  1.16915206 1.031154   1.14638846 1.06477704 0.98362414 1.06656043
  0.98243539 1.07852786 1.04075877 1.04218242]]
{0: 618, 1: 1254}
acc 0.452991452991453
(0.187

recall penalty
0 loss 141674.31104878703
dev set
[1.00122927 0.99808841 1.00014685 1.00000976 1.00111761 1.0004672
 1.00174587 1.00036581 1.00151716 1.00070107 0.99978236 1.00071998
 0.99977031 1.00083965 1.00046193 1.00047617]
[[1.11758189 0.81426829 1.00955973 1.00062431 1.10641144 1.04134324
  1.16925307 1.03120999 1.14637863 1.0647377  0.98362326 1.06662938
  0.98243468 1.07860295 1.040817   1.04224109]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

1 loss 141674.3109175712
dev set
[1.00128314 0.99803477 1.00019822 1.00001877 1.00117149 1.00052117
 1.00179968 1.00041971 1.001571   1.00075504 0.9997293  1.00077394
 0.99971696 1.00089359 1.0005159  1.00053015]
[[1.11762428 0.81432201 1.00955997 1.00062431 1.10645089 1.04135134
  1.16930252 1.03121408 1.14642591 1.06475853 0.98367603 1.06665129
  0.98248776 1.07863133 1.04082486 1.04224961]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667

In [53]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.35*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [64,128,256,512]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=7,debug=False)

[1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295.
 1295. 1295. 1295. 1295.]
batch-size: 64 alpha-init: 0.6
precision and recall penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 1249128.694508861
dev set
[0.59966419 0.59662467 0.59861625 0.59852027 0.60260125 0.60197537
 0.60325263 0.60187818 0.5999139  0.59909635 0.59828474 0.60192126
 0.59827266 0.60234796 0.60196187 0.60197611]
[[1.11906896 0.81573464 1.01104918 1.00212258 1.10485234 1.03977548
  1.16764946 1.02963845 1.14787911 1.06627299 0.98509706 1.06512126
  0.98392353 1.07701838 1.03926376 1.0406874 ]]
{0: 273, 1: 1599}
acc 0.27297008547008544
(0.14946841776110067, 0.9958333333333333, 0.25992387166938546, None)

1 loss 1249105.3682796431
dev set
[0.59817455 0.5950781  0.59712625 0.59703653 0.60412528 0.60350861
 0.6047882  0.60341265 0.59833777 0.59751978 0.59670813 0.60320008
 0.59669131 0.60388211 0.60353535 0.60354959]
[[1.1205784  0.81728596 1.01255367 1.00362401 1.10333518 1.03823884
  1.16611921 1.0281014  1.14945499 1.0678478  0.98666937 1.06359619
  0.9855036  1.07548098 1.03769335 1.039117  ]]
{0: 268, 1: 1604}
acc 0.2702991452991453
(0.14900249376558602, 0.9958333333333333, 0.25921908

2 loss 1249543.4740057117
dev set
[0.89686406 0.89385673 0.89604093 0.89562726 0.90517051 0.90481634
 0.90609142 0.90477846 0.89812447 0.89806577 0.89732951 0.90512769
 0.89978738 0.90519908 0.89621025 0.89622462]
[[1.12192346 0.81854792 1.01368241 1.00505982 1.10258779 1.0371544
  1.16502353 1.02690233 1.15019124 1.06774913 0.98652237 1.06203516
  0.98408554 1.07437959 1.04539245 1.04681608]]
{0: 1096, 1: 776}
acc 0.6677350427350427
(0.2538659793814433, 0.8208333333333333, 0.38779527559055116, None)

3 loss 1249542.4784437558
dev set
[0.89542646 0.89240643 0.89465887 0.89414737 0.9065363  0.90620549
 0.90752081 0.90617928 0.89699219 0.89723578 0.89649407 0.90659887
 0.89980941 0.90659292 0.89481121 0.89482563]
[[1.12340291 0.82003181 1.01510485 1.0065781  1.10132288 1.03586569
  1.16367465 1.02558889 1.15150136 1.06874282 0.98752949 1.06056064
  0.98463906 1.07308365 1.04693481 1.04835845]]
{0: 1092, 1: 780}
acc 0.6655982905982906
(0.25256410256410255, 0.8208333333333333, 0.3862745098

3 loss 648375.3450635042
dev set
[0.69806376 0.69502756 0.69704001 0.69693575 0.70414473 0.703504
 0.70479523 0.70340623 0.69831636 0.69750284 0.6966877  0.70373176
 0.69667247 0.7038776  0.70355254 0.70356678]
[[1.12067176 0.81733622 1.01263207 1.00371173 1.10330967 1.03824842
  1.1661095  1.02811396 1.14946976 1.06782621 0.98661658 1.06351688
  0.98551854 1.07549032 1.03767792 1.03910156]]
{0: 571, 1: 1301}
acc 0.42788461538461536
(0.180630284396618, 0.9791666666666666, 0.30499675535366644, None)

4 loss 648369.4702328773
dev set
[0.69728363 0.69423146 0.69626433 0.69616196 0.70490503 0.70426035
 0.70555883 0.70416164 0.69749868 0.69668732 0.69586403 0.7045086
 0.69585006 0.70463464 0.70436237 0.7043766 ]
[[1.12145915 0.81813597 1.01341457 1.00449279 1.10254961 1.03749041
  1.16534634 1.02735789 1.15028367 1.0686326  0.98741898 1.06273832
  0.98634253 1.07473177 1.03686726 1.03829091]]
{0: 557, 1: 1315}
acc 0.4204059829059829
(0.17870722433460076, 0.9791666666666666, 0.30225080385852

4 loss 648566.7356084513
dev set
[1.00714536 0.99331241 1.00652894 1.00580936 1.00708379 1.00674752
 1.00743369 1.00669108 1.00730382 1.00687136 0.99540994 1.00688102
 0.99523384 1.00694003 1.00674467 1.00675235]
[[1.12314746 0.81974503 1.01435943 1.0045387  1.11192759 1.04645318
  1.17494725 1.03623882 1.15203489 1.07001683 0.98886052 1.07192109
  0.98783219 1.08396949 1.04592288 1.04735793]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

batch-size: 256 alpha-init: 0.6
precision and recall penalty
0 loss 357989.688577729
dev set
[0.60076946 0.5977431  0.59969381 0.59961021 0.60146613 0.60081777
 0.60209597 0.60071671 0.60106089 0.6002446  0.59943306 0.60104369
 0.59942127 0.6011902  0.60081102 0.60082526]
[[1.11795067 0.81461379 1.00996168 1.00102149 1.10597633 1.04093407
  1.16880253 1.03080299 1.14673646 1.06512615 0.98397079 1.06621567
  0.9827831  1.07817711 1.04041018 1.04183383]]
{0: 284, 1: 1588}
acc 0.277777777777

precision and recall penalty
0 loss 358398.282760186
dev set
[0.90077641 0.89774972 0.89970991 0.899611   0.90145631 0.90081573
 0.90209463 0.90071554 0.90109914 0.90029144 0.89951496 0.90106535
 0.89972317 0.90118827 0.90001182 0.90002606]
[[1.11794436 0.81460741 1.00994638 1.00102014 1.10599284 1.04093873
  1.16880542 1.03080553 1.14672123 1.06509568 0.98391331 1.06621072
  0.98263787 1.07818152 1.04121648 1.04264013]]
{0: 1094, 1: 778}
acc 0.6666666666666666
(0.2532133676092545, 0.8208333333333333, 0.38703339882121807, None)

1 loss 358398.0083754707
dev set
[0.90038219 0.89736321 0.89932827 0.89922201 0.90184363 0.90121147
 0.9024896  0.90111135 0.90073558 0.89995607 0.89921031 0.90146055
 0.89972732 0.90158419 0.89961846 0.8996327 ]
[[1.11833964 0.81499476 1.01032949 1.00141015 1.10561368 1.04054709
  1.16841539 1.03041371 1.14710398 1.0654508  0.98424459 1.06581266
  0.98285833 1.0777895  1.04161592 1.04303956]]
{0: 1094, 1: 778}
acc 0.6666666666666666
(0.2532133676092545, 0.8208

1 loss 213187.355134262
dev set
[0.70075118 0.69772254 0.69967552 0.69958717 0.70148481 0.7008348
 0.70211423 0.70073425 0.70104329 0.70022714 0.69941578 0.70108584
 0.69940403 0.70120723 0.70082866 0.70084289]
[[1.11796882 0.81463425 1.0099799  1.00104371 1.10595702 1.04091717
  1.16878393 1.03078538 1.14675563 1.06514048 0.98398623 1.06619361
  0.98280133 1.0781602  1.0403922  1.04181585]]
{0: 610, 1: 1262}
acc 0.44871794871794873
(0.1862123613312203, 0.9791666666666666, 0.31291611185086554, None)

2 loss 213185.93578361443
dev set
[0.70054281 0.69751635 0.69946881 0.69938229 0.7016913  0.70104124
 0.7023213  0.70094127 0.70083627 0.7000203  0.699209   0.7012926
 0.69919711 0.70141369 0.7010357  0.70104994]
[[1.11817722 0.81484054 1.01018673 1.00124887 1.10575052 1.04071072
  1.16857688 1.03057813 1.14696307 1.06534504 0.98419098 1.06598683
  0.98300798 1.07795374 1.04018525 1.0416089 ]]
{0: 609, 1: 1263}
acc 0.4481837606837607
(0.18606492478226444, 0.9791666666666666, 0.312707917498

2 loss 213361.6980663713
dev set
[1.00183684 0.99750108 1.00074284 1.00026389 1.00172718 1.00109064
 1.00234625 1.00098934 1.00212035 1.00131984 0.99922132 1.0013383
 0.99919978 1.00145511 1.00108543 1.00109951]
[[1.1180018  0.81486655 1.00958875 1.00062461 1.10679919 1.04147337
  1.16976866 1.03130212 1.14686128 1.06496652 0.98420207 1.06686634
  0.98302032 1.07889027 1.04094504 1.04237479]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

3 loss 213361.69732003802
dev set
[1.0020809  0.99727903 1.00100826 1.00050728 1.00197327 1.0013515
 1.00258275 1.00125235 1.00235984 1.00157511 0.99900328 1.00159311
 0.99897778 1.00170707 1.00134642 1.00136017]
[[1.11822277 0.81510102 1.00965584 1.00062959 1.10701506 1.04161446
  1.17000038 1.03142254 1.14709014 1.06514557 0.98444548 1.06704785
  0.98326729 1.07908545 1.04108513 1.0425176 ]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.298534798534

In [52]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.35*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [64,128,256,512]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=11,debug=False)

[1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295.
 1295. 1295. 1295. 1295.]
batch-size: 64 alpha-init: 0.6
precision and recall and sign 2 penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 1249128.694508861
dev set
[0.59966419 0.59662467 0.59861625 0.59852027 0.60260125 0.60197537
 0.60325263 0.60187818 0.5999139  0.59909635 0.59828474 0.60192126
 0.59827266 0.60234796 0.60196187 0.60197611]
[[1.11906896 0.81573464 1.01104918 1.00212258 1.10485234 1.03977548
  1.16764946 1.02963845 1.14787911 1.06627299 0.98509706 1.06512126
  0.98392353 1.07701838 1.03926376 1.0406874 ]]
{0: 273, 1: 1599}
acc 0.27297008547008544
(0.14946841776110067, 0.9958333333333333, 0.25992387166938546, None)

1 loss 1249105.3682796431
dev set
[0.59817455 0.5950781  0.59712625 0.59703653 0.60412528 0.60350861
 0.6047882  0.60341265 0.59833777 0.59751978 0.59670813 0.60320008
 0.59669131 0.60388211 0.60353535 0.60354959]
[[1.1205784  0.81728596 1.01255367 1.00362401 1.10333518 1.03823884
  1.16611921 1.0281014  1.14945499 1.0678478  0.98666937 1.06359619
  0.9855036  1.07548098 1.03769335 1.039117  ]]
{0: 268, 1: 1604}
acc 0.2702991452991453
(0.14900249376558602, 0.9958333333333333, 0.25921908

1 loss 1249544.450272091
dev set
[0.89829558 0.89529817 0.8974117  0.8970978  0.90379912 0.90339646
 0.90465101 0.90334266 0.89924821 0.89890411 0.89817109 0.90364231
 0.89976149 0.90377473 0.89757531 0.89758963]
[[1.12045966 0.81708164 1.01228002 1.00356061 1.1038561  1.03847942
  1.16638376 1.02825904 1.14889482 1.06675641 0.98551947 1.06353646
  0.9835368  1.07571192 1.04386445 1.04528809]]
{0: 1095, 1: 777}
acc 0.6672008547008547
(0.25353925353925355, 0.8208333333333333, 0.38741396263520156, None)

2 loss 1249543.4740057117
dev set
[0.89686406 0.89385673 0.89604093 0.89562726 0.90517051 0.90481634
 0.90609142 0.90477846 0.89812447 0.89806577 0.89732951 0.90512769
 0.89978738 0.90519908 0.89621025 0.89622462]
[[1.12192346 0.81854792 1.01368241 1.00505982 1.10258779 1.0371544
  1.16502353 1.02690233 1.15019124 1.06774913 0.98652237 1.06203516
  0.98408554 1.07437959 1.04539245 1.04681608]]
{0: 1096, 1: 776}
acc 0.6677350427350427
(0.2538659793814433, 0.8208333333333333, 0.38779527559

2 loss 648381.0380018526
dev set
[0.69884163 0.69581761 0.69781132 0.69770699 0.70337999 0.70274093
 0.70402704 0.70264356 0.69912292 0.69830745 0.69749602 0.70295846
 0.69748212 0.70311401 0.70275214 0.70276638]
[[1.11988791 0.81654332 1.011855   1.00293447 1.104073   1.03901231
  1.16687638 1.02887668 1.14866729 1.06703248 0.9858282  1.06429436
  0.98470912 1.07625465 1.03847882 1.03990246]]
{0: 582, 1: 1290}
acc 0.4337606837606838
(0.1821705426356589, 0.9791666666666666, 0.30718954248366015, None)

3 loss 648375.3450635042
dev set
[0.69806376 0.69502756 0.69704001 0.69693575 0.70414473 0.703504
 0.70479523 0.70340623 0.69831636 0.69750284 0.6966877  0.70373176
 0.69667247 0.7038776  0.70355254 0.70356678]
[[1.12067176 0.81733622 1.01263207 1.00371173 1.10330967 1.03824842
  1.1661095  1.02811396 1.14946976 1.06782621 0.98661658 1.06351688
  0.98551854 1.07549032 1.03767792 1.03910156]]
{0: 571, 1: 1301}
acc 0.42788461538461536
(0.180630284396618, 0.9791666666666666, 0.30499675535366

3 loss 648566.7451122454
dev set
[1.00567756 0.99442171 1.00495125 1.00411927 1.00560592 1.00520813
 1.0060178  1.00514178 1.00586438 1.00535321 0.99643438 1.00536464
 0.99629199 1.00543559 1.00520478 1.00521382]
[[1.12153186 0.81840009 1.01266365 1.00277924 1.1103039  1.04478364
  1.1733746  1.03456234 1.15044244 1.06836265 0.98757914 1.0702682
  0.98651363 1.08232541 1.04425299 1.045689  ]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

4 loss 648566.7356084513
dev set
[1.00714536 0.99331241 1.00652894 1.00580936 1.00708379 1.00674752
 1.00743369 1.00669108 1.00730382 1.00687136 0.99540994 1.00688102
 0.99523384 1.00694003 1.00674467 1.00675235]
[[1.12314746 0.81974503 1.01435943 1.0045387  1.11192759 1.04645318
  1.17494725 1.03623882 1.15203489 1.07001683 0.98886052 1.07192109
  0.98783219 1.08396949 1.04592288 1.04735793]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.298534798534

4 loss 358341.39123283833
dev set
[0.79917717 0.79616738 0.79813819 0.79803837 0.80304887 0.8024006
 0.80368527 0.80230125 0.79961054 0.79962745 0.79839739 0.80264199
 0.79825405 0.80277347 0.80127696 0.80129002]
[[1.11954972 0.81619441 1.0115259  1.00259975 1.10440068 1.03935561
  1.16722003 1.02922292 1.14824834 1.06629145 0.98507963 1.06462309
  0.98391181 1.07659805 1.04263529 1.04405885]]
{0: 940, 1: 932}
acc 0.5982905982905983
(0.22532188841201717, 0.875, 0.3583617747440273, None)

batch-size: 256 alpha-init: 0.9
precision and recall and sign 2 penalty
0 loss 358398.282760186
dev set
[0.90077641 0.89774972 0.89970991 0.899611   0.90145631 0.90081573
 0.90209463 0.90071554 0.90109914 0.90029144 0.89951496 0.90106535
 0.89972317 0.90118827 0.90001182 0.90002606]
[[1.11794436 0.81460741 1.00994638 1.00102014 1.10599284 1.04093873
  1.16880542 1.03080553 1.14672123 1.06509568 0.98391331 1.06621072
  0.98263787 1.07818152 1.04121648 1.04264013]]
{0: 1094, 1: 778}
acc 0.666666666666666

precision and recall and sign 2 penalty
0 loss 213188.78584261215
dev set
[0.70096017 0.6979291  0.69988255 0.6997928  0.70127755 0.70062746
 0.70190631 0.70052635 0.70125037 0.70043408 0.69962264 0.70087881
 0.69961094 0.70099989 0.70062143 0.70063567]
[[1.11775985 0.81442762 1.00977278 1.00083785 1.10616422 1.04112449
  1.16899183 1.0309935  1.14654808 1.06493565 0.9837815  1.06640084
  0.98259474 1.07836753 1.0405993  1.04202295]]
{0: 614, 1: 1258}
acc 0.45085470085470086
(0.18680445151033387, 0.9791666666666666, 0.3137516688918558, None)

1 loss 213187.355134262
dev set
[0.70075118 0.69772254 0.69967552 0.69958717 0.70148481 0.7008348
 0.70211423 0.70073425 0.70104329 0.70022714 0.69941578 0.70108584
 0.69940403 0.70120723 0.70082866 0.70084289]
[[1.11796882 0.81463425 1.0099799  1.00104371 1.10595702 1.04091717
  1.16878393 1.03078538 1.14675563 1.06514048 0.98398623 1.06619361
  0.98280133 1.0781602  1.0403922  1.04181585]]
{0: 610, 1: 1262}
acc 0.44871794871794873
(0.18621236133

1 loss 213361.69871806965
dev set
[1.00160795 0.99771676 1.00050363 1.00008088 1.00149698 1.00085065
 1.00212211 1.00074824 1.00189435 1.00108352 0.99942983 1.00110229
 0.99941157 1.00122103 1.00084538 1.00085965]
[[1.117812   0.81464357 1.00956523 1.00062431 1.10661777 1.04138975
  1.16955828 1.03123884 1.14665722 1.06483822 0.98397802 1.0667348
  0.98279313 1.07874    1.04086249 1.04228936]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

2 loss 213361.6980663713
dev set
[1.00183684 0.99750108 1.00074284 1.00026389 1.00172718 1.00109064
 1.00234625 1.00098934 1.00212035 1.00131984 0.99922132 1.0013383
 0.99919978 1.00145511 1.00108543 1.00109951]
[[1.1180018  0.81486655 1.00958875 1.00062461 1.10679919 1.04147337
  1.16976866 1.03130212 1.14686128 1.06496652 0.98420207 1.06686634
  0.98302032 1.07889027 1.04094504 1.04237479]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.298534798534

In [51]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.35*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [64,128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=12,debug=False)

[1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295.
 1295. 1295. 1295. 1295.]
batch-size: 64 alpha-init: 0.6
recall and sign 2 penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 1242346.3576840137
dev set
[0.59966419 0.59662467 0.59861625 0.59852027 0.60260125 0.60197537
 0.60325263 0.60187818 0.5999139  0.59909635 0.59828474 0.60192128
 0.59827266 0.60234796 0.60196187 0.6019761 ]
[[1.11906896 0.81573464 1.01104918 1.00212258 1.10485233 1.03977548
  1.16764946 1.02963845 1.14787911 1.06627299 0.98509706 1.06512126
  0.98392353 1.07701838 1.03926376 1.0406874 ]]
{0: 273, 1: 1599}
acc 0.27297008547008544
(0.14946841776110067, 0.9958333333333333, 0.25992387166938546, None)

1 loss 1242323.0325994184
dev set
[0.59817455 0.5950781  0.59712625 0.59703654 0.60412528 0.60350861
 0.6047882  0.60341265 0.59833777 0.59751978 0.59670813 0.60320012
 0.59669131 0.60388211 0.60353535 0.60354959]
[[1.1205784  0.81728596 1.01255366 1.00362401 1.10333518 1.03823884
  1.16611921 1.0281014  1.14945499 1.0678478  0.98666936 1.06359618
  0.9855036  1.07548098 1.03769335 1.039117  ]]
{0: 268, 1: 1604}
acc 0.2702991452991453
(0.14900249376558602, 0.9958333333333333, 0.2592190

2 loss 1242761.0896406875
dev set
[0.8968642  0.89385698 0.89604117 0.89562734 0.90517048 0.90481632
 0.90609141 0.90477847 0.89812491 0.89806619 0.89732983 0.90512775
 0.89978897 0.90519906 0.89620948 0.89622385]
[[1.12192333 0.81854768 1.01368218 1.00505974 1.10258787 1.03715443
  1.16502358 1.02690231 1.15019108 1.06774886 0.98652217 1.06203516
  0.98408477 1.07437962 1.04539266 1.0468163 ]]
{0: 1096, 1: 776}
acc 0.6677350427350427
(0.2538659793814433, 0.8208333333333333, 0.38779527559055116, None)

3 loss 1242760.0940966338
dev set
[0.89542665 0.89240677 0.89465921 0.89414748 0.90653625 0.90620548
 0.90752079 0.9061793  0.89699278 0.89723634 0.8964945  0.90659894
 0.89981159 0.9065929  0.89481017 0.89482459]
[[1.12340274 0.82003148 1.01510454 1.00657799 1.10132299 1.03586572
  1.16367471 1.02558885 1.15150114 1.06874244 0.98752921 1.06056064
  0.984638   1.07308368 1.04693512 1.04835876]]
{0: 1092, 1: 780}
acc 0.6655982905982906
(0.25256410256410255, 0.8208333333333333, 0.386274509

3 loss 641722.9231608228
dev set
[0.69806376 0.69502756 0.69704002 0.69693575 0.70414473 0.703504
 0.70479523 0.70340623 0.69831636 0.69750284 0.6966877  0.70373177
 0.69667247 0.7038776  0.70355255 0.70356678]
[[1.12067176 0.81733623 1.01263207 1.00371173 1.10330967 1.03824842
  1.1661095  1.02811396 1.14946976 1.0678262  0.98661657 1.06351687
  0.98551854 1.07549032 1.03767792 1.03910157]]
{0: 571, 1: 1301}
acc 0.42788461538461536
(0.180630284396618, 0.9791666666666666, 0.30499675535366644, None)

4 loss 641717.0488815666
dev set
[0.69728363 0.69423146 0.69626433 0.69616196 0.70490503 0.70426035
 0.70555884 0.70416164 0.69749868 0.69668732 0.69586402 0.70450861
 0.69585005 0.70463464 0.70436237 0.70437661]
[[1.12145915 0.81813597 1.01341456 1.00449279 1.10254961 1.03749041
  1.16534633 1.02735789 1.15028367 1.06863259 0.98741897 1.06273831
  0.98634253 1.07473177 1.03686726 1.03829091]]
{0: 557, 1: 1315}
acc 0.4204059829059829
(0.17870722433460076, 0.9791666666666666, 0.3022508038585

...

1 loss 206619.9930347919
dev set
[0.70075118 0.69772254 0.69967552 0.69958717 0.70148481 0.7008348
 0.70211423 0.70073425 0.70104329 0.70022714 0.69941578 0.70108584
 0.69940403 0.70120723 0.70082866 0.70084289]
[[1.11796882 0.81463425 1.0099799  1.00104371 1.10595702 1.04091717
  1.16878393 1.03078538 1.14675563 1.06514047 0.98398623 1.06619361
  0.98280133 1.0781602  1.0403922  1.04181585]]
{0: 610, 1: 1262}
acc 0.44871794871794873
(0.1862123613312203, 0.9791666666666666, 0.31291611185086554, None)

2 loss 206618.57382909747
dev set
[0.70054281 0.69751635 0.69946881 0.69938229 0.7016913  0.70104124
 0.7023213  0.70094127 0.70083627 0.7000203  0.699209   0.7012926
 0.69919711 0.70141369 0.7010357  0.70104994]
[[1.11817722 0.81484054 1.01018673 1.00124887 1.10575052 1.04071072
  1.16857688 1.03057813 1.14696307 1.06534504 0.98419098 1.06598683
  0.98300798 1.07795374 1.04018525 1.0416089 ]]
{0: 609, 1: 1263}
acc 0.4481837606837607
(0.18606492478226444, 0.9791666666666666, 0.31270791749

2 loss 206794.30966232822
dev set
[1.00183684 0.99750087 1.00074284 1.00026389 1.00172718 1.00109064
 1.00234625 1.00098934 1.00212035 1.00131984 0.99922106 1.0013383
 0.99919961 1.00145511 1.00108543 1.00109951]
[[1.1180018  0.81486673 1.00958875 1.00062461 1.10679919 1.04147337
  1.16976866 1.03130212 1.14686128 1.06496652 0.98420175 1.06686634
  0.98301999 1.07889027 1.04094504 1.04237479]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

3 loss 206794.30892537007
dev set
[1.0020809  0.9972786  1.00100826 1.00050728 1.00197327 1.0013515
 1.00258275 1.00125235 1.00235984 1.00157511 0.99900273 1.00159311
 0.99897731 1.00170707 1.00134642 1.00136017]
[[1.11822277 0.81510139 1.00965584 1.00062959 1.10701506 1.04161446
  1.17000038 1.03142254 1.14709014 1.06514557 0.9844452  1.06704785
  0.98326701 1.07908545 1.04108513 1.0425176 ]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.29853479853

3 loss 123828.30931963224
dev set
[0.80074695 0.79771733 0.79967419 0.79958027 0.80148748 0.80083849
 0.80211875 0.80073837 0.80103893 0.80029666 0.79946437 0.80109074
 0.79941383 0.80121091 0.80065465 0.80066935]
[[1.11797298 0.81463944 1.0099815  1.00105031 1.10595459 1.04091328
  1.16877857 1.03078086 1.14675864 1.0651202  0.98395898 1.06618992
  0.98277166 1.07815633 1.04123552 1.04265916]]
{0: 958, 1: 914}
acc 0.6068376068376068
(0.2286652078774617, 0.8708333333333333, 0.3622183708838822, None)

4 loss 123828.1726350624
dev set
[0.80063995 0.79761118 0.79956892 0.7994739  0.80159291 0.80094433
 0.80222508 0.80084454 0.80093248 0.80021412 0.79937267 0.80119666
 0.79931079 0.80131675 0.80071435 0.80072916]
[[1.11807997 0.81474563 1.01008689 1.00115671 1.10584925 1.04080737
  1.168672   1.03067446 1.14686436 1.06521726 0.98405501 1.06608427
  0.98286773 1.07805043 1.04134009 1.04276373]]
{0: 957, 1: 915}
acc 0.6063034188034188
(0.2284153005464481, 0.8708333333333333, 0.36190476190476

4 loss 82034.19031644007
dev set
[0.60090586 0.59787461 0.59982752 0.59973783 0.60133206 0.60068165
 0.60196125 0.60058076 0.60119532 0.60037918 0.59956764 0.60093485
 0.59955574 0.60105408 0.60067624 0.60069048]
[[1.11781414 0.8144821  1.00982775 1.00089272 1.10610955 1.04107024
  1.16893652 1.03093897 1.14660299 1.06499148 0.98383839 1.06634667
  0.98264966 1.07831328 1.04054421 1.04196785]]
{0: 285, 1: 1587}
acc 0.2783119658119658
(0.14996849401386264, 0.9916666666666667, 0.26053639846743293, None)

batch-size: 2048 alpha-init: 0.7
recall and sign 2 penalty
0 loss 82301.91153231141
dev set
[0.70112151 0.69808853 0.70004192 0.69995252 0.70111753 0.70046712
 0.70174595 0.7003659  0.70140973 0.70059364 0.6997821  0.70071968
 0.69977022 0.70083955 0.70046178 0.70047602]
[[1.11759855 0.81426815 1.00961333 1.00067803 1.10632412 1.04128482
  1.16915206 1.031154   1.14638846 1.06477704 0.98362414 1.06656043
  0.98243539 1.07852786 1.04075877 1.04218242]]
{0: 618, 1: 1254}
acc 0.452991452991

recall and sign 2 penalty
0 loss 82474.31104878704
dev set
[1.00122927 0.99808841 1.00014685 1.00000976 1.00111761 1.0004672
 1.00174587 1.00036581 1.00151716 1.00070107 0.99978236 1.00071998
 0.99977031 1.00083965 1.00046193 1.00047617]
[[1.11758189 0.81426829 1.00955973 1.00062431 1.10641144 1.04134324
  1.16925307 1.03120999 1.14637863 1.0647377  0.98362326 1.06662938
  0.98243468 1.07860295 1.040817   1.04224109]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

1 loss 82474.31091757117
dev set
[1.00128314 0.99803477 1.00019822 1.00001877 1.00117149 1.00052117
 1.00179968 1.00041971 1.001571   1.00075504 0.9997293  1.00077394
 0.99971696 1.00089359 1.0005159  1.00053015]
[[1.11762428 0.81432201 1.00955997 1.00062431 1.10645089 1.04135134
  1.16930252 1.03121408 1.14642591 1.06475853 0.98367603 1.06665129
  0.98248776 1.07863133 1.04082486 1.04224961]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.679166

In [50]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.35*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [64,128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

[1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295.
 1295. 1295. 1295. 1295.]
batch-size: 64 alpha-init: 0.6
recall penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 1242346.3576840137
dev set
[0.59966419 0.59662467 0.59861625 0.59852027 0.60260125 0.60197537
 0.60325263 0.60187818 0.5999139  0.59909635 0.59828474 0.60192128
 0.59827266 0.60234796 0.60196187 0.6019761 ]
[[1.11906896 0.81573464 1.01104918 1.00212258 1.10485233 1.03977548
  1.16764946 1.02963845 1.14787911 1.06627299 0.98509706 1.06512126
  0.98392353 1.07701838 1.03926376 1.0406874 ]]
{0: 273, 1: 1599}
acc 0.27297008547008544
(0.14946841776110067, 0.9958333333333333, 0.25992387166938546, None)

1 loss 1242323.0325994184
dev set
[0.59817455 0.5950781  0.59712625 0.59703654 0.60412528 0.60350861
 0.6047882  0.60341265 0.59833777 0.59751978 0.59670813 0.60320012
 0.59669131 0.60388211 0.60353535 0.60354959]
[[1.1205784  0.81728596 1.01255366 1.00362401 1.10333518 1.03823884
  1.16611921 1.0281014  1.14945499 1.0678478  0.98666936 1.06359618
  0.9855036  1.07548098 1.03769335 1.039117  ]]
{0: 268, 1: 1604}
acc 0.2702991452991453
(0.14900249376558602, 0.9958333333333333, 0.2592190

2 loss 1242761.0896406875
dev set
[0.8968642  0.89385698 0.89604117 0.89562734 0.90517048 0.90481632
 0.90609141 0.90477847 0.89812491 0.89806619 0.89732983 0.90512775
 0.89978897 0.90519906 0.89620948 0.89622385]
[[1.12192333 0.81854768 1.01368218 1.00505974 1.10258787 1.03715443
  1.16502358 1.02690231 1.15019108 1.06774886 0.98652217 1.06203516
  0.98408477 1.07437962 1.04539266 1.0468163 ]]
{0: 1096, 1: 776}
acc 0.6677350427350427
(0.2538659793814433, 0.8208333333333333, 0.38779527559055116, None)

3 loss 1242760.0940966338
dev set
[0.89542665 0.89240677 0.89465921 0.89414748 0.90653625 0.90620548
 0.90752079 0.9061793  0.89699278 0.89723634 0.8964945  0.90659894
 0.89981159 0.9065929  0.89481017 0.89482459]
[[1.12340274 0.82003148 1.01510454 1.00657799 1.10132299 1.03586572
  1.16367471 1.02558885 1.15150114 1.06874244 0.98752921 1.06056064
  0.984638   1.07308368 1.04693512 1.04835876]]
{0: 1092, 1: 780}
acc 0.6655982905982906
(0.25256410256410255, 0.8208333333333333, 0.386274509

3 loss 641722.9231608228
dev set
[0.69806376 0.69502756 0.69704002 0.69693575 0.70414473 0.703504
 0.70479523 0.70340623 0.69831636 0.69750284 0.6966877  0.70373177
 0.69667247 0.7038776  0.70355255 0.70356678]
[[1.12067176 0.81733623 1.01263207 1.00371173 1.10330967 1.03824842
  1.1661095  1.02811396 1.14946976 1.0678262  0.98661657 1.06351687
  0.98551854 1.07549032 1.03767792 1.03910157]]
{0: 571, 1: 1301}
acc 0.42788461538461536
(0.180630284396618, 0.9791666666666666, 0.30499675535366644, None)

4 loss 641717.0488815666
dev set
[0.69728363 0.69423146 0.69626433 0.69616196 0.70490503 0.70426035
 0.70555884 0.70416164 0.69749868 0.69668732 0.69586402 0.70450861
 0.69585005 0.70463464 0.70436237 0.70437661]
[[1.12145915 0.81813597 1.01341456 1.00449279 1.10254961 1.03749041
  1.16534633 1.02735789 1.15028367 1.06863259 0.98741897 1.06273831
  0.98634253 1.07473177 1.03686726 1.03829091]]
{0: 557, 1: 1315}
acc 0.4204059829059829
(0.17870722433460076, 0.9791666666666666, 0.3022508038585

...

2 loss 206618.57382909747
dev set
[0.70054281 0.69751635 0.69946881 0.69938229 0.7016913  0.70104124
 0.7023213  0.70094127 0.70083627 0.7000203  0.699209   0.7012926
 0.69919711 0.70141369 0.7010357  0.70104994]
[[1.11817722 0.81484054 1.01018673 1.00124887 1.10575052 1.04071072
  1.16857688 1.03057813 1.14696307 1.06534504 0.98419098 1.06598683
  0.98300798 1.07795374 1.04018525 1.0416089 ]]
{0: 609, 1: 1263}
acc 0.4481837606837607
(0.18606492478226444, 0.9791666666666666, 0.31270791749833665, None)

3 loss 206617.14685267478
dev set
[0.70033445 0.69730988 0.69926184 0.69917743 0.70189754 0.7012473
 0.70252805 0.70114787 0.70062861 0.69981297 0.6990016  0.70149961
 0.69898948 0.70161976 0.70124322 0.70125746]
[[1.11838569 0.81504715 1.01039387 1.00145408 1.10554423 1.04050463
  1.16837012 1.03037128 1.1471711  1.06555016 0.98439645 1.06577996
  0.98321543 1.07774763 1.03997778 1.04140142]]
{0: 608, 1: 1264}
acc 0.44764957264957267
(0.18591772151898733, 0.9791666666666666, 0.3125, Non

3 loss 206794.30892537007
dev set
[1.0020809  0.9972786  1.00100826 1.00050728 1.00197327 1.0013515
 1.00258275 1.00125235 1.00235984 1.00157511 0.99900273 1.00159311
 0.99897731 1.00170707 1.00134642 1.00136017]
[[1.11822277 0.81510139 1.00965584 1.00062959 1.10701506 1.04161446
  1.17000038 1.03142254 1.14709014 1.06514557 0.9844452  1.06704785
  0.98326701 1.07908545 1.04108513 1.0425176 ]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

4 loss 206794.3080807947
dev set
[1.00234142 0.99704911 1.00129821 1.000792   1.00223645 1.00163337
 1.00283306 1.00153709 1.00261425 1.00184996 0.9987743  1.00186739
 0.99874426 1.00197778 1.00162843 1.00164177]
[[1.11847286 0.81534879 1.00978967 1.0006588  1.10726263 1.04181403
  1.17025362 1.03160619 1.14734322 1.06537131 0.98470742 1.06727517
  0.98353328 1.07932126 1.04128395 1.04271844]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.29853479853

4 loss 123828.1726350624
dev set
[0.80063995 0.79761118 0.79956892 0.7994739  0.80159291 0.80094433
 0.80222508 0.80084454 0.80093248 0.80021412 0.79937267 0.80119666
 0.79931079 0.80131675 0.80071435 0.80072916]
[[1.11807997 0.81474563 1.01008689 1.00115671 1.10584925 1.04080737
  1.168672   1.03067446 1.14686436 1.06521726 0.98405501 1.06608427
  0.98286773 1.07805043 1.04134009 1.04276373]]
{0: 957, 1: 915}
acc 0.6063034188034188
(0.2284153005464481, 0.8708333333333333, 0.361904761904762, None)

batch-size: 1024 alpha-init: 0.9
recall penalty
0 loss 123883.5187490397
dev set
[0.9010678  0.89803551 0.89998993 0.8998994  0.90117036 0.90052077
 0.90179942 0.90041952 0.90135712 0.90054062 0.89973571 0.90077313
 0.89978004 0.9008932  0.90030071 0.90031494]
[[1.11765226 0.81432119 1.00966541 1.00073117 1.10627158 1.04123108
  1.16909862 1.03110033 1.14644184 1.06483012 0.98367339 1.06650688
  0.98247511 1.07847413 1.04091984 1.04234348]]
{0: 1094, 1: 778}
acc 0.6666666666666666
(0.2532133

recall penalty
0 loss 82301.91153231141
dev set
[0.70112151 0.69808853 0.70004192 0.69995252 0.70111753 0.70046712
 0.70174595 0.7003659  0.70140973 0.70059364 0.6997821  0.70071968
 0.69977022 0.70083955 0.70046178 0.70047602]
[[1.11759855 0.81426815 1.00961333 1.00067803 1.10632412 1.04128482
  1.16915206 1.031154   1.14638846 1.06477704 0.98362414 1.06656043
  0.98243539 1.07852786 1.04075877 1.04218242]]
{0: 618, 1: 1254}
acc 0.452991452991453
(0.18740031897926634, 0.9791666666666666, 0.31459170013386883, None)

1 loss 82301.54742188164
dev set
[0.70106761 0.6980351  0.69998841 0.69989893 0.70117116 0.70052073
 0.70179979 0.70041963 0.70135631 0.70054024 0.6997287  0.70077298
 0.69971682 0.70089316 0.70051527 0.70052951]
[[1.11765244 0.81432159 1.00966685 1.00073162 1.10627049 1.04123122
  1.16909816 1.03110022 1.14644209 1.06483065 0.98367739 1.06650701
  0.98248871 1.07847426 1.04070522 1.04212886]]
{0: 617, 1: 1255}
acc 0.45245726495726496
(0.18725099601593626, 0.979166666666666

1 loss 82474.31091757117
dev set
[1.00128314 0.99803477 1.00019822 1.00001877 1.00117149 1.00052117
 1.00179968 1.00041971 1.001571   1.00075504 0.9997293  1.00077394
 0.99971696 1.00089359 1.0005159  1.00053015]
[[1.11762428 0.81432201 1.00955997 1.00062431 1.10645089 1.04135134
  1.16930252 1.03121408 1.14642591 1.06475853 0.98367603 1.06665129
  0.98248776 1.07863133 1.04082486 1.04224961]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.2985347985347986, None)

2 loss 82474.31078173517
dev set
[1.00133729 0.99798101 1.00024983 1.00003922 1.00122568 1.00057558
 1.0018537  1.00047402 1.00162507 1.0008094  0.99967608 1.00082829
 0.99966341 1.00094789 1.00057031 1.00058456]
[[1.11766942 0.81437594 1.00956057 1.00062431 1.10649361 1.04136291
  1.16935323 1.03122047 1.14647494 1.06478423 0.98372926 1.06667808
  0.98254134 1.07866433 1.04083613 1.04226169]]
{0: 1020, 1: 852}
acc 0.5908119658119658
(0.19131455399061034, 0.6791666666666667, 0.29853479853

In [48]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.35*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [64,128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

[1295. 1295. 1295. 1295. 1295. 1295. 1295. 1295.]
batch-size: 64 alpha-init: 0.6
recall penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 621458.5404233874
dev set
[0.59977473 0.59669853 0.59867669 0.60150101 0.59969289 0.60197459
 0.60321691 0.60183472]
[[1.11893333 0.81559947 1.01073512 0.99909126 1.10771657 1.03977863
  1.16771999 1.0297279 ]]
{0: 721, 1: 1151}
acc 0.4236111111111111
(0.1355343179843614, 0.65, 0.22429906542056074, None)

1 loss 621454.9790565936
dev set
[0.5981989  0.59510638 0.59709554 0.60301373 0.59811978 0.60352355
 0.60481623 0.60343445]
[[1.12049799 0.81715749 1.0121731  0.99754957 1.10927498 1.0382307
  1.16611668 1.02812487]]
{0: 675, 1: 1197}
acc 0.40544871794871795
(0.13533834586466165, 0.675, 0.2254697286012526, None)

2 loss 621450.675549065
dev set
[0.59646397 0.59337522 0.59536149 0.60452458 0.59636236 0.60505987
 0.60648478 0.60510577]
[[1.12221431 0.81888142 1.01383999 0.99600962 1.111028   1.03669508
  1.16440982 1.0264133 ]]
{0: 620, 1: 1252}
acc 0.38247863247863245
(0.134185303514377, 0.7, 0.225201072386059, None)

3 loss 621445.5550760408
dev set
[0.59459719 0.59152816 0.593

0 loss 321019.31571709225
dev set
[0.60045237 0.59740365 0.59936662 0.60077152 0.60035353 0.60119566
 0.60246023 0.60107947]
[[1.11826776 0.81493609 1.01019048 0.99985003 1.10707535 1.04055668
  1.16844947 1.03045326]]
{0: 737, 1: 1135}
acc 0.42895299145299143
(0.13480176211453745, 0.6375, 0.22254545454545455, None)

1 loss 321017.6456095978
dev set
[0.59969516 0.59663629 0.59860623 0.60154391 0.59960334 0.60197519
 0.60324329 0.60186228]
[[1.11902984 0.81569398 1.01087606 0.99907332 1.10781618 1.03977749
  1.16767032 1.02967501]]
{0: 718, 1: 1154}
acc 0.422008547008547
(0.13518197573656845, 0.65, 0.22381635581061693, None)

2 loss 321015.80276300356
dev set
[0.59889093 0.59582891 0.59780111 0.6023167  0.59879732 0.60275189
 0.60404491 0.60266435]
[[1.11983526 0.81650005 1.01163244 0.99829661 1.10861589 1.03900108
  1.1668625  1.02886655]]
{0: 695, 1: 1177}
acc 0.41399572649572647
(0.13593882752761258, 0.6666666666666666, 0.2258292166549047, None)

3 loss 321013.77324405377
dev set
[0.

0 loss 175979.69949663084
dev set
[0.60078995 0.59775262 0.59970918 0.60040537 0.60068282 0.60081734
 0.60209131 0.60071108]
[[1.11793291 0.81460117 1.00992103 1.00022268 1.10675601 1.04093469
  1.16881015 1.03081259]]
{0: 744, 1: 1128}
acc 0.4326923076923077
(0.1356382978723404, 0.6375, 0.2236842105263158, None)

1 loss 175978.86389964836
dev set
[0.60040571 0.59736431 0.59932393 0.60080286 0.60030274 0.60121612
 0.60248806 0.60110764]
[[1.11832207 0.81498607 1.01027977 0.9998245  1.1071329  1.04053601
  1.16841625 1.03041919]]
{0: 736, 1: 1136}
acc 0.4284188034188034
(0.13468309859154928, 0.6375, 0.22238372093023256, None)

2 loss 175977.99161700858
dev set
[0.60001013 0.59696638 0.59892786 0.60120099 0.59990901 0.60161448
 0.60288921 0.60150878]
[[1.11872162 0.81538236 1.01065558 0.99942599 1.107524   1.04013773
  1.16801559 1.03001864]]
{0: 724, 1: 1148}
acc 0.4252136752136752
(0.13588850174216027, 0.65, 0.22478386167146971, None)

3 loss 175977.0740979875
dev set
[0.59960243 0.596

0 loss 103459.89077783604
dev set
[0.60096703 0.59793245 0.59988671 0.60021915 0.60085681 0.60062739
 0.60190461 0.60052456]
[[1.11775379 0.814423   1.00975866 1.00041074 1.10658452 1.04112457
  1.16899447 1.03099649]]
{0: 752, 1: 1120}
acc 0.43696581196581197
(0.13660714285714284, 0.6375, 0.22499999999999998, None)

1 loss 103459.45450880057
dev set
[0.60076487 0.59772888 0.59968401 0.60042662 0.60065614 0.60083477
 0.60211097 0.60073086]
[[1.11795781 0.81462462 1.00994883 1.00020326 1.1067847  1.04091721
  1.16878942 1.03079162]]
{0: 744, 1: 1128}
acc 0.4326923076923077
(0.1356382978723404, 0.6375, 0.2236842105263158, None)

2 loss 103459.01550800775
dev set
[0.60056118 0.59752415 0.59947989 0.60063367 0.60045346 0.6010414
 0.60231748 0.60093734]
[[1.11816328 0.81482768 1.01014156 0.99999635 1.10698696 1.0407106
  1.16858383 1.03058615]]
{0: 741, 1: 1131}
acc 0.4310897435897436
(0.13527851458885942, 0.6375, 0.22319474835886216, None)

3 loss 103458.56666828283
dev set
[0.60035494 0.5

0 loss 62019.98598110012
dev set
[0.60107049 0.59803661 0.59999039 0.60011317 0.60095942 0.60052055
 0.60179872 0.60041872]
[[1.11765    0.81431941 1.00966084 1.00051723 1.10648201 1.0412314
  1.1690997  1.03110162]]
{0: 756, 1: 1116}
acc 0.4391025641025641
(0.13709677419354838, 0.6375, 0.22566371681415928, None)

1 loss 62019.7663454671
dev set
[0.60096717 0.59793259 0.59988677 0.60021918 0.60085685 0.60062647
 0.60190412 0.60052408]
[[1.11775378 0.81442244 1.00975956 1.00041133 1.10658406 1.04112549
  1.16899482 1.03099682]]
{0: 752, 1: 1120}
acc 0.43696581196581197
(0.13660714285714284, 0.6375, 0.22499999999999998, None)

2 loss 62019.54604718163
dev set
[0.60086363 0.59782843 0.59978296 0.60032512 0.60075398 0.60073224
 0.6020095  0.60062943]
[[1.1178577  0.81452563 1.0098587  1.00030556 1.10668635 1.04101975
  1.1688899  1.03089198]]
{0: 751, 1: 1121}
acc 0.43643162393162394
(0.13648528099910795, 0.6375, 0.22483468038207202, None)

3 loss 62019.32348671989
dev set
[0.60075968 0.59

recall penalty
0 loss 41300.03305860571
dev set
[0.60112238 0.59808893 0.60004247 0.60005996 0.60101084 0.60046712
 0.60174562 0.60036563]
[[1.11759798 0.81426765 1.00961175 1.00057062 1.1064308  1.04128482
  1.16915254 1.03115441]]
{0: 759, 1: 1113}
acc 0.4407051282051282
(0.13746630727762804, 0.6375, 0.2261640798226164, None)

1 loss 41299.92245157751
dev set
[0.60106979 0.59803613 0.5999898  0.60011355 0.60095846 0.60052073
 0.60179895 0.60041895]
[[1.117651   0.81432032 1.00966279 1.00051708 1.10648315 1.04123122
  1.16909936 1.03110126]]
{0: 756, 1: 1116}
acc 0.4391025641025641
(0.13709677419354838, 0.6375, 0.22566371681415928, None)

2 loss 41299.811719297795
dev set
[0.60101723 0.59798335 0.59993715 0.60016712 0.6009061  0.60057432
 0.60185228 0.60047226]
[[1.11770402 0.81437296 1.00971375 1.00046356 1.10653548 1.04117764
  1.16904619 1.03104812]]
{0: 755, 1: 1117}
acc 0.43856837606837606
(0.1369740376007162, 0.6375, 0.22549742078113483, None)

3 loss 41299.70046810007
dev set
[

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import numpy as np
def get_LF_rec(L_S,true_labels):
    #L_S : a numpy array of [NoOfDataPoints,2,NoOfLFs] 
    #true_labels : numpy array [NoOfDataPoints]
    
    true_l = [-1 if x==0 else x for x in true_labels]
    unique, counts = np.unique(true_l, return_counts=True)
    print(dict(zip(unique, counts)))

    # take only labels 
    L_S = L_S[:,0,:]
    #L_S shape [NoOfDataPoints,NoOfLFs]
    LF_rec = []
    for i in range(L_S.shape[1]):
#         print(accuracy_score(L_S[:,i],tl,normalize=False),accuracy_score(L_S[:,i],tl))
       
        LF_labels = [LF_l[i] if x==LF_l[i] else 0 for x in L_S[:,i]]
        tl = [LF_l[i] if x==LF_l[i] else 0 for x in true_l]
        LF_rec.append(recall_score(LF_labels,tl,pos_label=LF_l[i],average='binary'))
#         unique, counts = np.unique(L_S[:,i], return_counts=True)
#         print(i,dict(zip(unique, counts)))
#         print(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
#         LF_acc.append(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
    return np.array(LF_rec)
                      
get_LF_rec(dev_L_S,gold_labels_dev)

{1: 240, -1: 1632}


array([0.22666667, 0.1953602 , 0.30890052, 0.55192878, 0.44444444,
       1.        , 0.55      , 1.        ])

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import numpy as np
def get_LF_acc(L_S,true_labels):
    #L_S : a numpy array of [NoOfDataPoints,2,NoOfLFs] 
    #true_labels : numpy array [NoOfDataPoints]
    
    tl = [-1 if x==0 else x for x in true_labels]
    unique, counts = np.unique(tl, return_counts=True)
    print(dict(zip(unique, counts)))

    # take only labels 
    L_S = L_S[:,0,:]
    #L_S shape [NoOfDataPoints,NoOfLFs]
    LF_acc = []
    for i in range(L_S.shape[1]):
#         print(accuracy_score(L_S[:,i],tl,normalize=False),accuracy_score(L_S[:,i],tl))
        LF_acc.append(accuracy_score(L_S[:,i],tl))
#         unique, counts = np.unique(L_S[:,i], return_counts=True)
#         print(i,dict(zip(unique, counts)))
#         print(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
#         LF_acc.append(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
    return np.array(LF_acc)
                      
get_LF_acc(dev_L_S,gold_labels_dev)

{1: 240, -1: 1632}


array([0.04540598, 0.08547009, 0.06303419, 0.09935897, 0.0042735 ,
       0.00053419, 0.00587607, 0.00106838])

In [35]:
rec = np.array([0.22666667, 0.1953602 , 0.30890052, 0.55192878, 0.44444444,\
       1.        , 0.55      , 1.        ])

acc = np.array([0.04540598, 0.08547009, 0.06303419, 0.09935897, 0.0042735 ,\
       0.00053419, 0.00587607, 0.00106838])
### smooth LFs with acc on discrete LFs
for b in [512,1024,2048]:
    for i in np.linspace(0,1,11):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

batch-size: 512 alpha-init: 0.0
precision and recall penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 19496.0543522807
dev set
[ 0.00096141 -0.0020719  -0.00011844 -0.0002078   0.00084974  0.00062756
  0.00190611  0.00052612]
[[1.11775864 0.81442855 1.0097736  1.00083837 1.10659187 1.04112433
  1.16899194 1.03099379]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 19486.788160600034
dev set
[ 0.00075394 -0.00227937 -0.00032591 -0.00041527  0.00064222  0.00083499
  0.00211357  0.00073359]
[[1.11796616 0.81463598 1.00998106 1.0010458  1.10679934 1.04091686
  1.16878442 1.03078627]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2 loss 19477.659813126596
dev set
[ 0.0005471  -0.00248622 -0.00053276 -0.00062211  0.00043527  0.00104159
  0.00232039  0.0009404 ]
[[1.11817307 0.8148428  1.01018791 1.0012526  1.10700625 1.04071018
  1.16857755 1.0305794 ]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

3 loss 19468.547479154557
dev set
[ 0.0003403  -0.00269303 -

acc 0.19123931623931623
(0.13432835820895522, 0.975, 0.23612512613521697, None)

1 loss 21715.31998849169
dev set
[0.50075514 0.49772175 0.49967488 0.49958571 0.50064347 0.50083486
 0.50211283 0.50073283]
[[1.11796521 0.81463464 1.00997912 1.00020333 1.10679794 1.04091706
  1.16878523 1.0307871 ]]
{0: 122, 1: 1750}
acc 0.18696581196581197
(0.1337142857142857, 0.975, 0.2351758793969849, None)

2 loss 21713.55875179972
dev set
[0.50054829 0.4975149  0.49946777 0.49937881 0.50043652 0.50104148
 0.50231968 0.50093968]
[[1.11817235 0.8148413  1.01018573 0.99999604 1.10700478 1.04071042
  1.16857838 1.03058025]]
{0: 120, 1: 1752}
acc 0.1858974358974359
(0.13356164383561644, 0.975, 0.23493975903614456, None)

3 loss 21711.78605284642
dev set
[0.50034088 0.49730753 0.49926015 0.4991714  0.5002289  0.50124786
 0.50252687 0.50114687]
[[1.11838008 0.81504854 1.01039299 0.99978899 1.10721231 1.040504
  1.16837117 1.03037303]]
{0: 117, 1: 1755}
acc 0.1842948717948718
(0.13333333333333333, 0.975, 0.

1 loss 21652.065804020564
dev set
[1.00160795 0.99773616 1.00050388 1.00010058 1.00149697 1.00085065
 1.00212211 1.00074823]
[[1.11781206 0.81462414 1.00956526 1.00062431 1.10661769 1.04138967
  1.16955825 1.03123878]]
{0: 1819, 1: 53}
acc 0.8605769230769231
(0.3018867924528302, 0.06666666666666667, 0.10921501706484642, None)

2 loss 21652.06565001153
dev set
[1.00183685 0.99753712 1.00074312 1.00029233 1.00172718 1.00109064
 1.00234625 1.00098933]
[[1.1180019  0.8148307  1.00958886 1.00062479 1.10679908 1.04147321
  1.16976861 1.03130198]]
{0: 1819, 1: 53}
acc 0.8605769230769231
(0.3018867924528302, 0.06666666666666667, 0.10921501706484642, None)

3 loss 21652.06549042031
dev set
[1.00208091 0.99733655 1.00100853 1.00053923 1.00197326 1.00135151
 1.00258274 1.00125235]
[[1.11822289 0.81504406 1.00965612 1.00063125 1.10701492 1.04161421
  1.17000032 1.0314223 ]]
{0: 1819, 1: 53}
acc 0.8605769230769231
(0.3018867924528302, 0.06666666666666667, 0.10921501706484642, None)

4 loss 21652.06

1 loss 21486.048109984997
dev set
[0.40096245 0.39792914 0.39988259 0.39979324 0.40085078 0.40062658
 0.40190511 0.40052512]
[[1.11775763 0.81442747 1.00977265 1.00083739 1.10659079 1.04112533
  1.16899291 1.03099476]]
{0: 8, 1: 1864}
acc 0.13247863247863248
(0.12875536480686695, 1.0, 0.22813688212927757, None)

2 loss 21484.18004111199
dev set
[0.40085667 0.39782335 0.39977681 0.39968746 0.400745   0.4007324
 0.4020109  0.40063091]
[[1.11786342 0.81453322 1.00987849 1.00094321 1.10669652 1.04101951
  1.1688871  1.03088895]]
{0: 8, 1: 1864}
acc 0.13247863247863248
(0.12875536480686695, 1.0, 0.22813688212927757, None)

3 loss 21482.310336153525
dev set
[0.40075087 0.39771755 0.39967102 0.39958166 0.40063919 0.40083815
 0.40211669 0.4007367 ]
[[1.11796923 0.81463899 1.00998435 1.00104907 1.10680228 1.04091374
  1.16878128 1.03078314]]
{0: 8, 1: 1864}
acc 0.13247863247863248
(0.12875536480686695, 1.0, 0.22813688212927757, None)

4 loss 21480.437427829
dev set
[0.40064502 0.3976117  0.3995

1 loss 21604.743389136256
dev set
[0.9013904  0.89834368 0.90030821 0.90021933 0.90127562 0.90062609
 0.90147848 0.90009849]
[[1.11733078 0.81401641 1.00934787 1.00041119 1.10617134 1.04112618
  1.16941958 1.03142143]]
{0: 1743, 1: 129}
acc 0.8317307692307693
(0.20930232558139536, 0.1125, 0.14634146341463414, None)

2 loss 21604.648755641618
dev set
[0.90149799 0.89844237 0.90041341 0.90032506 0.90138093 0.90073164
 0.9013724  0.8999924 ]
[[1.11722357 0.81391973 1.00924331 1.00030546 1.10606897 1.04102082
  1.16952568 1.03152753]]
{0: 1743, 1: 129}
acc 0.8317307692307693
(0.20930232558139536, 0.1125, 0.14634146341463414, None)

3 loss 21604.554007786875
dev set
[0.90160568 0.89854096 0.90051856 0.9004307  0.90148629 0.90083713
 0.90126633 0.89988632]
[[1.11711621 0.81382318 1.00913878 1.00019983 1.10596653 1.04091555
  1.16963179 1.03163364]]
{0: 1743, 1: 129}
acc 0.8317307692307693
(0.20930232558139536, 0.1125, 0.14634146341463414, None)

4 loss 21604.459065203948
dev set
[0.9017135  

1 loss 21098.119539319276
dev set
[0.30106811 0.29803481 0.29998826 0.29989891 0.30095645 0.3005208
 0.3017994  0.30041942]
[[1.117652   0.81432188 1.00966696 1.00073166 1.10648519 1.04123113
  1.16909861 1.03110047]]
{0: 4, 1: 1868}
acc 0.13034188034188035
(0.1284796573875803, 1.0, 0.22770398481973433, None)

2 loss 21096.723732367725
dev set
[0.30101452 0.29798122 0.29993467 0.29984532 0.30090286 0.30057443
 0.301853   0.30047302]
[[1.11770561 0.81437548 1.00972053 1.00078527 1.10653878 1.04117748
  1.16904499 1.03104684]]
{0: 4, 1: 1868}
acc 0.13034188034188035
(0.1284796573875803, 1.0, 0.22770398481973433, None)

3 loss 21095.327453786653
dev set
[0.30096092 0.29792762 0.29988107 0.29979172 0.30084926 0.30062806
 0.3019066  0.30052662]
[[1.11775923 0.81442909 1.00977411 1.00083889 1.10659237 1.04112384
  1.16899136 1.03099321]]
{0: 4, 1: 1868}
acc 0.13034188034188035
(0.1284796573875803, 1.0, 0.22770398481973433, None)

4 loss 21093.93051554237
dev set
[0.30090732 0.29787401 0.2998

1 loss 21534.92252826949
dev set
[0.801283   0.79824965 0.8002036  0.80011347 0.80117131 0.80052054
 0.80158442 0.80020444]
[[1.11743682 0.81410692 1.00945173 1.00051704 1.10627027 1.04123152
  1.1693137  1.03131555]]
{0: 1580, 1: 292}
acc 0.7692307692307693
(0.17123287671232876, 0.20833333333333334, 0.1879699248120301, None)

2 loss 21534.77264992377
dev set
[0.80133673 0.79830336 0.80025761 0.80016699 0.80122503 0.80057401
 0.80153063 0.80015065]
[[1.11738293 0.81405316 1.00939777 1.00046349 1.10621651 1.04117813
  1.16936753 1.03136938]]
{0: 1580, 1: 292}
acc 0.7692307692307693
(0.17123287671232876, 0.20833333333333334, 0.1879699248120301, None)

3 loss 21534.62263213058
dev set
[0.80139047 0.79835707 0.80031161 0.8002205  0.80127875 0.80062749
 0.80147683 0.80009685]
[[1.11732904 0.81399939 1.00934382 1.00040995 1.10616275 1.04112474
  1.16942136 1.03142321]]
{0: 1580, 1: 292}
acc 0.7692307692307693
(0.17123287671232876, 0.20833333333333334, 0.1879699248120301, None)

4 loss 21534.

In [39]:
for b in [512,1024,2048]:
    for i in np.linspace(0,1,11):
        print("batch-size:",b,"alpha-init:",i)
        train(0.001,5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=True,penalty=4)

batch-size: 512 alpha-init: 0.0
{1: 240, -1: 1632}
k Tensor("Const:0", shape=(16,), dtype=float64)
s Tensor("unstack:1", shape=(?, 16), dtype=float64)
l Tensor("unstack:0", shape=(?, 16), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 16), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 16), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(16,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(16,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(16,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(16,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision penalty
sft aj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArra

/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 44803.77891834587
dev set
[-0.00665541 -0.00972818 -0.00773116 -0.00778921  0.008907    0.00826529
  0.00955428  0.00816225 -0.00644768 -0.00726407 -0.00807588 -0.00724516
 -0.00810952  0.00863976  0.00831868  0.00833292]
[[1.12545668 0.82209959 1.01743631 1.00849136 1.09851205 1.03345878
  1.16132077 1.02332928 1.15424913 1.07263723 0.99148243 1.07452679
  0.99031014 1.0707008  1.03289795 1.03432159]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 44397.08388221004
dev set
[-0.01263042 -0.01721241 -0.0147579  -0.01409974  0.01606824  0.0153725
  0.01676995  0.01526962 -0.01406356 -0.01488173 -0.01569542 -0.01486277
 -0.01586948  0.01576635  0.01593224  0.0159465 ]
[[1.13261492 0.82968062 1.02487486 1.01560181 1.09115944 1.02613475
  1.15392608 1.0159989  1.16189607 1.08027784 0.99912521 1.08214521
  0.99804704 1.06336646 1.0252608  1.02668444]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2

0 loss 44803.77891834587
dev set
[-0.00665541 -0.00972818 -0.00773116 -0.00778921  0.008907    0.00826529
  0.00955428  0.00816225 -0.00644768 -0.00726407 -0.00807588 -0.00724516
 -0.00810952  0.00863976  0.00831868  0.00833292]
[[1.12545668 0.82209959 1.01743631 1.00849136 1.09851205 1.03345878
  1.16132077 1.02332928 1.15424913 1.07263723 0.99148243 1.07452679
  0.99031014 1.0707008  1.03289795 1.03432159]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 44397.08388221004
dev set
[-0.01263042 -0.01721241 -0.0147579  -0.01409974  0.01606824  0.0153725
  0.01676995  0.01526962 -0.01406356 -0.01488173 -0.01569542 -0.01486277
 -0.01586948  0.01576635  0.01593224  0.0159465 ]
[[1.13261492 0.82968062 1.02487486 1.01560181 1.09115944 1.02613475
  1.15392608 1.0159989  1.16189607 1.08027784 0.99912521 1.08214521
  0.99804704 1.06336646 1.0252608  1.02668444]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2

0 loss 44803.77891834587
dev set
[-0.00665541 -0.00972818 -0.00773116 -0.00778921  0.008907    0.00826529
  0.00955428  0.00816225 -0.00644768 -0.00726407 -0.00807588 -0.00724516
 -0.00810952  0.00863976  0.00831868  0.00833292]
[[1.12545668 0.82209959 1.01743631 1.00849136 1.09851205 1.03345878
  1.16132077 1.02332928 1.15424913 1.07263723 0.99148243 1.07452679
  0.99031014 1.0707008  1.03289795 1.03432159]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 44397.08388221004
dev set
[-0.01263042 -0.01721241 -0.0147579  -0.01409974  0.01606824  0.0153725
  0.01676995  0.01526962 -0.01406356 -0.01488173 -0.01569542 -0.01486277
 -0.01586948  0.01576635  0.01593224  0.0159465 ]
[[1.13261492 0.82968062 1.02487486 1.01560181 1.09115944 1.02613475
  1.15392608 1.0159989  1.16189607 1.08027784 0.99912521 1.08214521
  0.99804704 1.06336646 1.0252608  1.02668444]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2

...

0 loss 44924.19311714624
dev set
[-0.00082067 -0.00384316 -0.00189225 -0.00198236  0.00305     0.00240009
  0.00368088  0.00229901 -0.00052501 -0.00134114 -0.00215268 -0.00132223
 -0.00216559  0.00277257  0.00239645  0.00241069]
[[1.11954196 0.81620043 1.01154899 1.00261495 1.10439074 1.03935068
  1.1672158  1.02921969 1.14832357 1.06671197 0.98555899 1.06860262
  0.984371   1.07659371 1.03882359 1.04024723]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 44814.91539254971
dev set
[-0.00279845 -0.00582005 -0.00386776 -0.00395418  0.00502459  0.00437477
  0.00565972  0.00427391 -0.00250794 -0.00332411 -0.00413569 -0.0033052
 -0.00415193  0.00474752  0.00437928  0.00439351]
[[1.12153145 0.81817933 1.01353081 1.00459675 1.10241275 1.03737203
  1.165233   1.02724066 1.15030763 1.06869569 0.98754225 1.07058593
  0.98635666 1.07461491 1.03683964 1.03826329]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2

prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f_p Tensor("map/while/Sum_1:0", shape=(), dtype=float64)
sft_p Tensor("map/while/sft_p:0", shape=(), dtype=float64)
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 44924.19311714624
dev set
[-0.00082067 -0.00384316 -0.00189225 -0.00198236  0.00305     0.00240009
  0.00368088  0.00229901 -0.00052501 -0.00134114 -0.00215268 -0.00132223
 -0.00216559  0.00277257  0.00239645  0.00241069]
[[1.11954196 0.81620043 1.01154899 1.00261495 1.10439074 1.03935068
  1.1672158  1.02921969 1.14832357 1.06671197 0.98555899 1.06860262
  0.984371   1.07659371 1.03882359 1.04024723]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 44814.91539254971
dev set
[-0.00279845 -0.00582005 -0.00386776 -0.00395418  0.00502459  0.00437477
  0.00565972  0.00427391 -0.00250794 

0 loss 44924.19311714624
dev set
[-0.00082067 -0.00384316 -0.00189225 -0.00198236  0.00305     0.00240009
  0.00368088  0.00229901 -0.00052501 -0.00134114 -0.00215268 -0.00132223
 -0.00216559  0.00277257  0.00239645  0.00241069]
[[1.11954196 0.81620043 1.01154899 1.00261495 1.10439074 1.03935068
  1.1672158  1.02921969 1.14832357 1.06671197 0.98555899 1.06860262
  0.984371   1.07659371 1.03882359 1.04024723]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 44814.91539254971
dev set
[-0.00279845 -0.00582005 -0.00386776 -0.00395418  0.00502459  0.00437477
  0.00565972  0.00427391 -0.00250794 -0.00332411 -0.00413569 -0.0033052
 -0.00415193  0.00474752  0.00437928  0.00439351]
[[1.12153145 0.81817933 1.01353081 1.00459675 1.10241275 1.03737203
  1.165233   1.02724066 1.15030763 1.06869569 0.98754225 1.07058593
  0.98635666 1.07461491 1.03683964 1.03826329]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2

loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 44924.19311714624
dev set
[-0.00082067 -0.00384316 -0.00189225 -0.00198236  0.00305     0.00240009
  0.00368088  0.00229901 -0.00052501 -0.00134114 -0.00215268 -0.00132223
 -0.00216559  0.00277257  0.00239645  0.00241069]
[[1.11954196 0.81620043 1.01154899 1.00261495 1.10439074 1.03935068
  1.1672158  1.02921969 1.14832357 1.06671197 0.98555899 1.06860262
  0.984371   1.07659371 1.03882359 1.04024723]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 44814.91539254971
dev set
[-0.00279845 -0.00582005 -0.00386776 -0.00395418  0.00502459  0.00437477
  0.00565972  0.00427391 -0.00250794 -0.00332411 -0.00413569 -0.0033052
 -0.00415193  0.00474752  0.00437928  0.00439351]
[[1.12153145 0.81817933 1.01353081 1.00459675 1.10241275 1.03737203
  1.165233   1.02724066 1.15030763 1.06869569 0.98754225 1.07058593
  0.98635666 1.07461491 

0 loss 44924.19311714624
dev set
[-0.00082067 -0.00384316 -0.00189225 -0.00198236  0.00305     0.00240009
  0.00368088  0.00229901 -0.00052501 -0.00134114 -0.00215268 -0.00132223
 -0.00216559  0.00277257  0.00239645  0.00241069]
[[1.11954196 0.81620043 1.01154899 1.00261495 1.10439074 1.03935068
  1.1672158  1.02921969 1.14832357 1.06671197 0.98555899 1.06860262
  0.984371   1.07659371 1.03882359 1.04024723]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 44814.91539254971
dev set
[-0.00279845 -0.00582005 -0.00386776 -0.00395418  0.00502459  0.00437477
  0.00565972  0.00427391 -0.00250794 -0.00332411 -0.00413569 -0.0033052
 -0.00415193  0.00474752  0.00437928  0.00439351]
[[1.12153145 0.81817933 1.01353081 1.00459675 1.10241275 1.03737203
  1.165233   1.02724066 1.15030763 1.06869569 0.98754225 1.07058593
  0.98635666 1.07461491 1.03683964 1.03826329]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2

In [34]:
for b in [32,64,128,512,1024,2048]:
    for i in np.linspace(0,1,11):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=True,penalty=4)

batch-size: 32 alpha-init: 0.0
{1: 240, -1: 1632}
k Tensor("Const:0", shape=(8,), dtype=float64)
s Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(8,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(8,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision penalty
sft aj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:

/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 26816.959090156495
dev set
[-0.00195583 -0.00498997 -0.00303594 -0.00312532 -0.00207781  0.00353034
  0.00482125  0.00344105]
[[1.1206748  0.81734738 1.01268406 1.00375999 1.10951252 1.03821237
  1.16607478 1.0280766 ]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 26679.22348918799
dev set
[-0.00508447 -0.00812134 -0.0061654  -0.00625486 -0.00524505  0.00656156
  0.00794032  0.00655939]
[[1.12381086 0.82048341 1.01581863 1.00689397 1.11267264 1.03515752
  1.16295096 1.02495257]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2 loss 26541.46427210567
dev set
[-0.0082123  -0.01125385 -0.00929465 -0.00938424 -0.00843865  0.00951716
  0.01105199  0.00966984]
[[1.12695172 0.82362449 1.01895851 1.01003062 1.11585588 1.03215973
  1.15983201 1.02183327]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

3 loss 26403.833156552482
dev set
[-0.01133781 -0.01438592 -

sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tensor("map/while/Gather_2:0", shape=(?, 8), dtype=float64)
out Tensor("map/while/prec_zy/while/Select:0", shape=(8,), dtype=float64)
ky Tensor("map/while/prec_zy/while/Select:0", shape=(8,), dtype=float64)
intsy Tensor("map/while/prec_zy/while/add:0", shape=(8,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f_p Tensor("map/while/Sum_1:0", shape=(), dtype=float64)
sft_p Tensor("map/while/sft_p:0", shape=(), dtype=float64)
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 26816.959090156495
dev set
[-0.00195583 -0.00498997 -0.00303594 -0.00312532 -0.00207781  0.00353034
  0.00482125  

2 loss 26541.46427210567
dev set
[-0.0082123  -0.01125385 -0.00929465 -0.00938424 -0.00843865  0.00951716
  0.01105199  0.00966984]
[[1.12695172 0.82362449 1.01895851 1.01003062 1.11585588 1.03215973
  1.15983201 1.02183327]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

3 loss 26403.833156552482
dev set
[-0.01133781 -0.01438592 -0.01242216 -0.01251191 -0.01165561  0.01239712
  0.01415546  0.01277158]
[[1.13009562 0.82676887 1.02210156 1.01316836 1.11905927 1.02921759
  1.15671872 1.0187195 ]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

4 loss 26266.41167972165
dev set
[-0.01446006 -0.01751655 -0.01554696 -0.01563692 -0.01489355  0.01520105
  0.01725044  0.01586434]
[[1.13324142 0.82991541 1.02524653 1.01630609 1.12228065 1.0263301
  1.1536115  1.01561168]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

batch-size: 32 alpha-init: 0.6000000000000001
{1: 240, -1: 16

...

out Tensor("map/while/prec_zy/while/Select:0", shape=(8,), dtype=float64)
ky Tensor("map/while/prec_zy/while/Select:0", shape=(8,), dtype=float64)
intsy Tensor("map/while/prec_zy/while/add:0", shape=(8,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f_p Tensor("map/while/Sum_1:0", shape=(), dtype=float64)
sft_p Tensor("map/while/sft_p:0", shape=(), dtype=float64)
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 26719.8664999681
dev set
[ 1.06826185e-03 -1.96503963e-03 -1.15880820e-05 -1.00941571e-04
  9.56594252e-04  5.20615353e-04  1.79924654e-03  4.19264920e-04]
[[1.11765181 0.81432171 1.00966683 1.0007315  1.10648504 1.04123131
  1.1690988  1.03110065]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 26715.17295831645
dev set
[ 0.00096229 -0.00207101 -0.00011756 -0.000206

3 loss 26711.06669404441
dev set
[ 0.00096085 -0.00207246 -0.000119   -0.00020836  0.00084917  0.00062808
  0.00190666  0.00052668]
[[1.11775928 0.81442915 1.00977423 1.00083893 1.10659246 1.04112382
  1.16899133 1.03099318]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

4 loss 26708.70458664201
dev set
[ 0.00090723 -0.00212608 -0.00017262 -0.00026198  0.00079555  0.00068172
  0.00196028  0.0005803 ]
[[1.11781291 0.81448278 1.00982784 1.00089256 1.10664609 1.04107017
  1.1689377  1.03093955]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

batch-size: 2048 alpha-init: 0.2
{1: 240, -1: 1632}
k Tensor("Const:0", shape=(8,), dtype=float64)
s Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorAr

intsy Tensor("map/while/prec_zy/while/add:0", shape=(8,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f_p Tensor("map/while/Sum_1:0", shape=(), dtype=float64)
sft_p Tensor("map/while/sft_p:0", shape=(), dtype=float64)
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 26718.156258829244
dev set
[ 1.12169996e-03 -1.91160120e-03  4.18501418e-05 -4.75033377e-05
  1.01003262e-03  4.67151725e-04  1.74580724e-03  3.65825700e-04]
[[1.11759838 0.81426828 1.00961339 1.00067806 1.10643161 1.04128478
  1.16915224 1.03115409]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 26715.790929390085
dev set
[ 1.06807656e-03 -1.96522493e-03 -1.17733775e-05 -1.01126867e-04
  9.56408981e-04  5.20803939e-04  1.79943191e-03  4.19450283e-04]
[[1.11765202 0.81432191 1.00966701 1.00073169 1.10648523 1.

3 loss 26711.06669404441
dev set
[ 0.00096085 -0.00207246 -0.000119   -0.00020836  0.00084917  0.00062808
  0.00190666  0.00052668]
[[1.11775928 0.81442915 1.00977423 1.00083893 1.10659246 1.04112382
  1.16899133 1.03099318]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

4 loss 26708.70458664201
dev set
[ 0.00090723 -0.00212608 -0.00017262 -0.00026198  0.00079555  0.00068172
  0.00196028  0.0005803 ]
[[1.11781291 0.81448278 1.00982784 1.00089256 1.10664609 1.04107017
  1.1689377  1.03093955]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

batch-size: 2048 alpha-init: 0.7000000000000001
{1: 240, -1: 1632}
k Tensor("Const:0", shape=(8,), dtype=float64)
s Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("

0 loss 26718.156258829244
dev set
[ 1.12169996e-03 -1.91160120e-03  4.18501418e-05 -4.75033377e-05
  1.01003262e-03  4.67151725e-04  1.74580724e-03  3.65825700e-04]
[[1.11759838 0.81426828 1.00961339 1.00067806 1.10643161 1.04128478
  1.16915224 1.03115409]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

1 loss 26715.790929390085
dev set
[ 1.06807656e-03 -1.96522493e-03 -1.17733775e-05 -1.01126867e-04
  9.56408981e-04  5.20803939e-04  1.79943191e-03  4.19450283e-04]
[[1.11765202 0.81432191 1.00966701 1.00073169 1.10648523 1.04123112
  1.1690986  1.03110045]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051282, 1.0, 0.22727272727272727, None)

2 loss 26713.428718701245
dev set
[ 1.01446164e-03 -2.01884029e-03 -6.53884454e-05 -1.54741947e-04
  9.02792473e-04  5.74445341e-04  1.85304779e-03  4.73066049e-04]
[[1.11770565 0.81437553 1.00972062 1.00078531 1.10653885 1.04117746
  1.16904497 1.03104682]]
{1: 1872}
acc 0.1282051282051282
(0.1282051282051

In [19]:
for b in [32,64,128,512,1024,2048]:
    print("batch-size:",b)
    train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False,penalty=4)

batch-size: 32
{1: 240, -1: 1632}
Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(8,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tenso

1 loss 22617.4147115639
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[1.11832614 0.81575247 1.01097787 1.00183516 1.10481415 1.03977321
  1.16764016 1.02964262]]
{0: 760, 1: 1112}
acc 0.5235042735042735
(0.2068345323741007, 0.9583333333333334, 0.34023668639053256, None)

2 loss 22612.28296817468
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[1.11869122 0.81652352 1.01167164 1.00241922 1.10403408 1.03899299
  1.1668594  1.02886235]]
{0: 760, 1: 1112}
acc 0.5235042735042735
(0.2068345323741007, 0.9583333333333334, 0.34023668639053256, None)

3 loss 22607.19584584605
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[1.1190219  0.81729352 1.01235629 1.00298637 1.10325541 1.03821436
  1.16608016 1.02808369]]
{0: 760

sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tensor("map/while/Gather_2:0", shape=(?, 8), dtype=float64)
out Tensor("map/while/prec_zy/while/Select:0", shape=(8,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f Tensor("map/while/Sum_1:0", shape=(), dtype=float64)
sft Tensor("map/while/sft:0", shape=(), dtype=float64)
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 22533.376976482286
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[1.11759798 0.81426819 1.00961346 1.00067826 1

In [20]:
for b in [32,64,128,512,1024,2048]:
    print("batch-size:",b)
    train(0.01,5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False,penalty=4)

batch-size: 32
{1: 240, -1: 1632}
Tensor("unstack:1", shape=(?, 8), dtype=float64)
l Tensor("unstack:0", shape=(?, 8), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 8), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 8), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(8,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(8,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tenso

1 loss 21202.920851586525
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[0.58858906 1.19849737 0.65724342 0.52894557 0.66488086 0.61575753
  0.71529883 0.60853572]]
{0: 759, 1: 1113}
acc 0.5229700854700855
(0.20664869721473494, 0.9583333333333334, 0.33998521803399856, None)

2 loss 20822.772916188456
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[0.38656065 1.30212164 0.48185169 0.32502514 0.53188935 0.50201065
  0.56908658 0.49774713]]
{0: 759, 1: 1113}
acc 0.5229700854700855
(0.20664869721473494, 0.9583333333333334, 0.33998521803399856, None)

3 loss 20682.11495647486
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[0.26060603 1.36562896 0.37911686 0.19353206 0.44963199 0.44009768
  0.47218621 0.43881049]]
{0

prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f Tensor("map/while/Sum_1:0", shape=(), dtype=float64)
sft Tensor("map/while/sft:0", shape=(), dtype=float64)
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 22504.5093901236
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[1.13603094 0.8339986  1.02917033 1.01976874 1.08653959 1.02150288
  1.14935453 1.0113711 ]]
{0: 760, 1: 1112}
acc 0.5235042735042735
(0.2068345323741007, 0.9583333333333334, 0.34023668639053256, None)

1 loss 22387.40116251994
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04]
[[1.12639968 0.85354162 1.04699709 1.02918603 1.06688159 1.00185859
  1.12967054 0.99172674]]
{0: 760, 1: 1112}
acc 0.

In [261]:
import scipy.sparse as sp
import _pickle as pkl
# L_train = pkl.load(open("train_L_S_discrete.p","rb"))
# L_train = sp.csr_matrix(L_train)

# L_gold = pkl.load(open("gold_discrete.p","rb"))
# print(np.array(L_gold).shape)
# L_gold = sp.csr_matrix(L_gold)

L_train = np.load("train_L_S_discrete.npy")
L_train = L_train[:,0,:].astype(int)
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)

L_gold = np.load("test_L_S_discrete.npy")
L_gold = L_gold[:,0,:].astype(int)
print(np.array(L_gold).shape)
L_gold = sp.csr_matrix(L_gold)

from snorkel.learning import GenerativeModel
import time
import datetime

gen_model = GenerativeModel()

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, epochs = 100, cardinality=2)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)


print("trained in ",str(datetime.timedelta(seconds=time.time() - start_time)))

(3700, 5)
(1872, 5)
started at: 9-6-2018, 20:22:49
trained in  0:00:06.133447


In [262]:
# 5 LFs
import numpy as np
dev_marginals = gen_model.marginals(L_gold)
dev_marginals = np.array(dev_marginals)
print(dev_marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in dev_marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(true_labels),GenLabels,average="binary"))


(1872,)
(1872,)
(0.18799212598425197, 0.7958333333333333, 0.3041401273885351, None)


In [219]:
# 8 discrete LFs
import numpy as np
dev_marginals = gen_model.marginals(L_gold)
dev_marginals = np.array(dev_marginals)
print(dev_marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in dev_marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(true_labels),GenLabels,average="binary"))


(1872,)
(1872,)
(0.20664869721473494, 0.9583333333333334, 0.33998521803399856, None)
